In [ ]:
####  All the imports  ####

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import logging
import time
from scipy.sparse import csr_matrix

import networkx as nx
import pickle
from concurrent.futures import ThreadPoolExecutor
from sklearn.neighbors import NearestNeighbors

from torch.utils.data import Dataset
from torch.utils.data import TensorDataset, DataLoader

# file_path = '/kaggle/input/recohit-dataset-100/RecoOutPileup_TimeMod_uniform_1_100recohitfile.tsv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model_path = "/kaggle/input/new-model-4dim/trained_full_model.pt"

import networkx as nx
from joblib import Parallel, delayed

import torch.optim as optim

from torch.utils.data import random_split

from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
folder_path = '/kaggle/working/embed_data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    
model_dir = "/kaggle/working/model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the figures in the "plots" folder
plots_dir = "/kaggle/working/plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale

# test_embedded_points
import os

# Check if the directory already exists.
if not os.path.exists("/kaggle/working/experiment_dir"):

    os.makedirs("/kaggle/working/experiment_dir")
experiment_dir = "/kaggle/working/experiment_dir"

# Create the "embed_data" folder if it doesn't exist
folder_path = '/kaggle/working/embed_data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    
# model_path = '/kaggle/input/model-new-4d/trained_full_model.pt'
# if not os.path.exists(model_path):
#     os.makedirs(model_path)
    
# Save the figures in the "plots" folder
plots_dir = "/kaggle/working/plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)
    
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
experiment_dir = '/kaggle/working/'

import torch
# from torch_geometric.nn import GCNConv
import networkx as nx
import numpy as np
from sklearn.neighbors import KDTree
import time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.cluster import DBSCAN
from collections import defaultdict
import time

import torch
from torch.utils.data import DataLoader, TensorDataset
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import networkx as nx
from collections import defaultdict
import pickle

In [ ]:
!pip install torch_geometric

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch
import torch.nn as nn
import time
import os
import logging
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code] {"jupyter":{"outputs_hidden":false}}
#Import all the libraries
#Install trackml, run these 3 seperately




# %% [code] {"jupyter":{"outputs_hidden":false}}
#defining the class


#####################################
#               DATASET             #
#####################################

class Hit_Pair_Dataset(Dataset):
    def __init__(self, data_filepath, nb_samples):
        dataset = pd.read_csv("/kaggle/input/new-set/pairwise_data_normalized.csv")

        try:
            self.hits_a = np.array(dataset[['pos_x1', 'pos_y1', 'pos_z1', 'time_ns1']][:nb_samples], dtype=np.float32)
            self.hits_b = np.array(dataset[['pos_x2', 'pos_y2', 'pos_z2', 'time_ns2']][:nb_samples], dtype=np.float32)
            self.target = np.array(dataset['label'][:nb_samples], dtype=np.float32)
            self.time1 = np.array(dataset['time_ns1'][:nb_samples], dtype=np.float32)
            self.muon1 = np.array(dataset['track_no1'][:nb_samples], dtype=np.float32)
        
        except:
            dim = (dataset.shape[1] - 1) // 2
            self.hits_a = dataset[:nb_samples, :dim]
            self.hits_b = dataset[:nb_samples, dim:2 * dim]
            self.target = dataset[:nb_samples]
            self.time1 = dataset[:nb_samples]
            self.muon1 = dataset[:nb_samples]
            

    def __getitem__(self, index):
        h_a = self.hits_a[index]
        h_b = self.hits_b[index]
        t = self.target[index]
        ti = self.time1[index]
        mi = self.muon1[index]
        return h_a, h_b, t, ti, mi

    def __len__(self):
        return len(self.hits_a)

    def get_dim(self):
        return self.hits_a.shape[1]

# %% [code] {"jupyter":{"outputs_hidden":false}}
dataset = Hit_Pair_Dataset("/kaggle/input/new-set/pairwise_data_normalized.csv", nb_samples=30000)


class MLP(nn.Module):
    def __init__(self,
                 nb_hidden,
                 nb_layer,
                 input_dim,
                 mean,
                 std,
                 emb_dim=4):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, nb_hidden)]
        ln = [nn.Linear(nb_hidden, nb_hidden) for _ in range(nb_layer-1)]
        layers.extend(ln)
        self.layers = nn.ModuleList(layers)
        self.emb_layer = nn.Linear(nb_hidden, emb_dim)
        self.act = nn.ReLU()
        self.mean = torch.FloatTensor(mean).to(torch.float)
        self.std = torch.FloatTensor(std).to(torch.float)

    def forward(self, hits):
        hits = self.normalize(hits)
        for l in self.layers:
            hits = l(hits)
            hits = self.act(hits)
            # hits = self.dropout(hits)
        hits = self.emb_layer(hits)
        return hits

    def normalize(self, hits):

        return hits

mlp = MLP(nb_hidden=128, nb_layer=3, input_dim=4, mean=[0,0,0,0], std=[1,1,1,1])
print("MLP model:", mlp)

# %% [code] {"jupyter":{"outputs_hidden":false}}
#score calculation: correctly predicted labels/total labels

def score_dist_accuracy(pred, true):
    pred_classes = torch.ones_like(pred)  # Initialize all predictions as 1
    pred_classes[pred >= 0.1] = -1 
    correct = pred_classes == true
    nb_correct = correct.sum().item()
    nb_total = true.size(0)
    score = nb_correct / nb_total
    return score

# %% [code] {"jupyter":{"outputs_hidden":false}}
####################
#     PRINTING     #
####################
def print_header():
  '''
  Print header before train / evaluation run.
  '''
  logging.info("         Loss  Score")

def print_eval_stats(nb_processed, loss, score):
  '''
  Log stats during train, evaluation run.
  '''
  logging.info("  {:5d}: {:.3f}  {:2.2f}".format(nb_processed, loss, score))
    
#spliting the dataset for training and evaluation

from torch.utils.data import random_split

import torch
from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split


# Split the dataset into training and validation subsets
train_ratio = 0.8
valid_ratio = 0.2
train_size = int(train_ratio * len(dataset))
valid_size = len(dataset) - train_size

# train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_dataset, valid_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


# Create data loaders for training and validation
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

# %% [code] {"jupyter":{"outputs_hidden":false}}

# Check if a GPU is available, otherwise use CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = dataset.get_dim()
net = MLP(nb_hidden=128, nb_layer=3, input_dim=4, mean=[0,0,0,0], std=[1,1,1,1])
net.to(DEVICE)

# %% [code] {"jupyter":{"outputs_hidden":false}}
#defining the model for training one epoch


model_dir = "/kaggle/working/model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


def train_one_epoch(net, batch_size, optimizer, train_loader):
  net.train()

  nb_batch = len(train_loader)
  nb_train = nb_batch * batch_size
  epoch_score = 0
  epoch_loss  = 0


  logging.info("Training on {} samples".format(nb_train))
  print_header()
  t0 = time.time()
  elapsed = 0
  for i, (hits_a, hits_b, target, time1, muon1) in enumerate(train_loader):
    hits_a = hits_a.to(DEVICE, non_blocking=True)
    hits_b = hits_b.to(DEVICE, non_blocking=True)
    target = target.to(DEVICE, non_blocking=True)
    time1 = time1.to(DEVICE, non_blocking=True)
    muon1 = muon1.to(DEVICE, non_blocking=True)
    '''
    hits_a = hits_a.to(DEVICE)
    hits_b = hits_b.to(DEVICE)
    target = target.to(DEVICE)
    '''
    optimizer.zero_grad()

    emb_h_a = net(hits_a)
    emb_h_b = net(hits_b)

    pred = nn.functional.pairwise_distance(emb_h_a,emb_h_b)
    true_dist = target
    loss = nn.functional.hinge_embedding_loss(pred,true_dist)
    
    torch.nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')

    loss.backward()
    optimizer.step()

    score = score_dist_accuracy(pred, target)
    epoch_score += score * 100
    epoch_loss  += loss.item()

    nb_proc = (i+1) * batch_size
    if (((i+0) % (nb_batch//10)) == 0):
        print_eval_stats(nb_proc,
                               epoch_loss/(i+1),
                               epoch_score/(i+1))

  logging.info("Model elapsed:  {:.2f}".format(elapsed))

  # Save the trained model
  torch.save(net.state_dict(), os.path.join(model_dir, "trained_model.pt"))

  return epoch_loss / nb_batch, epoch_score / nb_batch

#   Save the trained model
# torch.save(net.state_dict(), os.path.join(model_dir, "trained_model.pt"))


def evaluate(net, batch_size, valid_loader):
    net.eval()

    nb_batch = len(valid_loader)
    nb_valid = nb_batch * batch_size
    valid_score = 0
    valid_loss = 0

    logging.info("Evaluating on {} samples".format(nb_valid))
    print_header()
    t0 = time.time()
    elapsed = 0
    with torch.no_grad():
        for i, (hits_a, hits_b, target, time1, muon1) in enumerate(valid_loader):
            hits_a = hits_a.to(DEVICE, non_blocking=True)
            hits_b = hits_b.to(DEVICE, non_blocking=True)
            target = target.to(DEVICE, non_blocking=True)
            time1 = time1.to(DEVICE, non_blocking=True)
            muon1 = muon1.to(DEVICE, non_blocking=True)

            emb_h_a = net(hits_a)
            emb_h_b = net(hits_b)

            pred = nn.functional.pairwise_distance(emb_h_a, emb_h_b)
            true_dist = target
            loss = nn.functional.hinge_embedding_loss(pred, true_dist)

            score = score_dist_accuracy(pred, target)
            valid_score += score * 100
            valid_loss += loss.item()

            nb_proc = (i + 1) * batch_size
            if (((i + 0) % (nb_batch // 10)) == 0):
                print_eval_stats(nb_proc, valid_loss / (i + 1), valid_score / (i + 1))
                
            # Save the evaluated model
            torch.save(net.state_dict(), os.path.join(model_dir, "evaluated_model.pt"))

    logging.info("Model elapsed:  {:.2f}".format(elapsed))

    return valid_loss / nb_batch, valid_score / nb_batch
#   Save the evaluated model

batch_size = 512

import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net.to(DEVICE)

# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# optimizer = optim.SGD(net.parameters(), lr=0.001)


# Train the model
train_losses=[]
train_scores=[]
valid_losses = []
valid_scores = []

num_epochs = 35
for epoch in range(num_epochs):
    epoch_loss, epoch_score = train_one_epoch(net, batch_size, optimizer, train_loader)
    train_losses.append(epoch_loss)
    train_scores.append(epoch_score)
    valid_loss, valid_score = evaluate(net, batch_size, valid_loader)
    valid_losses.append(valid_loss)
    valid_scores.append(valid_score)
#     print(f"Epoch {epoch+1}: Loss: {epoch_loss}, Score: {epoch_score}")
    print(f"Epoch {epoch+1}: Train Loss: {epoch_loss}, Train Score: {epoch_score}, Valid Loss: {valid_loss}, Valid Score: {valid_score}")

# %% [code] {"jupyter":{"outputs_hidden":false}}
# Plot train loss and valid loss in one plot

# Save the figures in the "plots" folder
plots_dir = "/kaggle/working/plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Save the Train Loss vs. Valid Loss plot
loss_plot_filename = os.path.join(plots_dir, "train_valid_loss_plot.png")

#Plot loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, num_epochs + 1), valid_losses, label='Valid Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss vs. Valid Loss')
plt.legend()
plt.grid()
plt.savefig(loss_plot_filename)
plt.show()


# Save the Train Accuracy vs. Valid Accuracy plot
accuracy_plot_filename = os.path.join(plots_dir, "train_valid_accuracy_plot.png")
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_scores, label='Train Accuracy', marker='o')
plt.plot(range(1, num_epochs + 1), valid_scores, label='Valid Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train Accuracy vs. Valid Accuracy')
plt.legend()
plt.grid()
plt.savefig(accuracy_plot_filename)
# Close the figure to free up memory
plt.show()

# %% [code] {"jupyter":{"outputs_hidden":false}}
#defining all the necessary functions

def track_epoch_stats(epoch_nb, lrate, train_stats, val_stats, experiment_dir):
    print("Epoch: {}".format(epoch_nb))
    print("Learning rate: {:.3g}".format(lrate))
    print("Train loss: {:.3f}".format(train_stats[0]))
    print("Train score: {:.2f}".format(train_stats[1]))
    print("Validation loss: {:.3f}".format(val_stats[0]))
    print("Validation score: {:.2f}".format(val_stats[1]))
    print()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch
import torch.nn as nn
import time
import os
import logging
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader


import os




#####################################
#               DATASET             #
#####################################

class Hit_Pair_Dataset(Dataset):
    def __init__(self, data_filepath, nb_samples):
        dataset = pd.read_csv("/kaggle/input/new-set/pairwise_data_normalized.csv")

        try:
            self.hits_a = np.array(dataset[['pos_x1', 'pos_y1', 'pos_z1', 'time_ns1']][:nb_samples], dtype=np.float32)
            self.hits_b = np.array(dataset[['pos_x2', 'pos_y2', 'pos_z2', 'time_ns2']][:nb_samples], dtype=np.float32)
            self.target = np.array(dataset['label'][:nb_samples], dtype=np.float32)
            
            self.muon1 = np.array(dataset['track_no1'][:nb_samples], dtype=np.float32)
        
        except:
            dim = (dataset.shape[1] - 1) // 2
            self.hits_a = dataset[:nb_samples, :dim]
            self.hits_b = dataset[:nb_samples, dim:2 * dim]
            self.target = dataset[:nb_samples]
            
            self.muon1 = dataset[:nb_samples]
            

    def __getitem__(self, index):
        h_a = self.hits_a[index]
        h_b = self.hits_b[index]
        t = self.target[index]
        
        mi = self.muon1[index]
        return h_a, h_b, t, mi

    def __len__(self):
        return len(self.hits_a)

    def get_dim(self):
        return self.hits_a.shape[1]

# %% [code] {"jupyter":{"outputs_hidden":false}}
dataset = Hit_Pair_Dataset("/kaggle/input/new-set/pairwise_data_normalized.csv", nb_samples=30000)


class MLP(nn.Module):
    def __init__(self,
                 nb_hidden,
                 nb_layer,
                 input_dim,
                 mean,
                 std,
                 emb_dim=4):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, nb_hidden)]
        ln = [nn.Linear(nb_hidden, nb_hidden) for _ in range(nb_layer-1)]
        layers.extend(ln)
        self.layers = nn.ModuleList(layers)
        self.emb_layer = nn.Linear(nb_hidden, emb_dim)
        self.act = nn.ReLU()
        self.mean = torch.FloatTensor(mean).to(torch.float)
        self.std = torch.FloatTensor(std).to(torch.float)

    def forward(self, hits):
        hits = self.normalize(hits)
        for l in self.layers:
            hits = l(hits)
            hits = self.act(hits)
            # hits = self.dropout(hits)
        hits = self.emb_layer(hits)
        return hits

    def normalize(self, hits):

        return hits

# mlp = MLP(nb_hidden=256, nb_layer=4, input_dim=4, mean=[0,0,0,0], std=[1,1,1,1])
# print("MLP model:", mlp)


def score_dist_accuracy(pred, true):
    pred_classes = torch.ones_like(pred)  # Initialize all predictions as 1
    pred_classes[pred >= 0.1] = -1 
    correct = pred_classes == true
    nb_correct = correct.sum().item()
    nb_total = true.size(0)
    score = nb_correct / nb_total
    return score


####################
#     PRINTING     #
####################
def print_header():
  '''
  Print header before train / evaluation run.
  '''
  logging.info("         Loss  Score")

def print_eval_stats(nb_processed, loss, score):
  '''
  Log stats during train, evaluation run.
  '''
  logging.info("  {:5d}: {:.3f}  {:2.2f}".format(nb_processed, loss, score))
    
#spliting the dataset for training and evaluation

from torch.utils.data import random_split

import torch
from torch.utils.data import DataLoader


from sklearn.model_selection import train_test_split


# Split the dataset into training and validation subsets
train_ratio = 0.8
valid_ratio = 0.2
train_size = int(train_ratio * len(dataset))
valid_size = len(dataset) - train_size

# train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_dataset, valid_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


# Create data loaders for training and validation
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

# %% [code] {"jupyter":{"outputs_hidden":false}}

# Check if a GPU is available, otherwise use CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = dataset.get_dim()
net = MLP(nb_hidden=64, nb_layer=3, input_dim=4, mean=[0,0,0,0], std=[1,1,1,1])
net.to(DEVICE)

# %% [code] {"jupyter":{"outputs_hidden":false}}
#defining the model for training one epoch


model_dir = "/kaggle/working/model"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


def train_one_epoch(net, batch_size, optimizer, train_loader):
  net.train()

  nb_batch = len(train_loader)
  nb_train = nb_batch * batch_size
  epoch_score = 0
  epoch_loss  = 0


  logging.info("Training on {} samples".format(nb_train))
  print_header()
  t0 = time.time()
  elapsed = 0
  for i, (hits_a, hits_b, target,  muon1) in enumerate(train_loader):
    hits_a = hits_a.to(DEVICE, non_blocking=True)
    hits_b = hits_b.to(DEVICE, non_blocking=True)
    target = target.to(DEVICE, non_blocking=True)
    
    muon1 = muon1.to(DEVICE, non_blocking=True)
    '''
    hits_a = hits_a.to(DEVICE)
    hits_b = hits_b.to(DEVICE)
    target = target.to(DEVICE)
    '''
    optimizer.zero_grad()

    emb_h_a = net(hits_a)
    emb_h_b = net(hits_b)

    pred = nn.functional.pairwise_distance(emb_h_a,emb_h_b)
    true_dist = target
    loss = nn.functional.hinge_embedding_loss(pred,true_dist)
    
    torch.nn.HingeEmbeddingLoss(margin=1.0, size_average=None, reduce=None, reduction='mean')

    loss.backward()
    optimizer.step()

    score = score_dist_accuracy(pred, target)
    epoch_score += score * 100
    epoch_loss  += loss.item()

    nb_proc = (i+1) * batch_size
    if (((i+0) % (nb_batch//10)) == 0):
        print_eval_stats(nb_proc,
                               epoch_loss/(i+1),
                               epoch_score/(i+1))

  logging.info("Model elapsed:  {:.2f}".format(elapsed))

  # Save the trained model
  torch.save(net.state_dict(), os.path.join(model_dir, "trained_model1.pt"))

  return epoch_loss / nb_batch, epoch_score / nb_batch

#   Save the trained model
# torch.save(net.state_dict(), os.path.join(model_dir, "trained_model.pt"))


def evaluate(net, batch_size, valid_loader):
    net.eval()

    nb_batch = len(valid_loader)
    nb_valid = nb_batch * batch_size
    valid_score = 0
    valid_loss = 0

    logging.info("Evaluating on {} samples".format(nb_valid))
    print_header()
    t0 = time.time()
    elapsed = 0
    with torch.no_grad():
        for i, (hits_a, hits_b, target,  muon1) in enumerate(valid_loader):
            hits_a = hits_a.to(DEVICE, non_blocking=True)
            hits_b = hits_b.to(DEVICE, non_blocking=True)
            target = target.to(DEVICE, non_blocking=True)
            
            muon1 = muon1.to(DEVICE, non_blocking=True)

            emb_h_a = net(hits_a)
            emb_h_b = net(hits_b)

            pred = nn.functional.pairwise_distance(emb_h_a, emb_h_b)
            true_dist = target
            loss = nn.functional.hinge_embedding_loss(pred, true_dist)

            score = score_dist_accuracy(pred, target)
            valid_score += score * 100
            valid_loss += loss.item()

            nb_proc = (i + 1) * batch_size
            if (((i + 0) % (nb_batch // 10)) == 0):
                print_eval_stats(nb_proc, valid_loss / (i + 1), valid_score / (i + 1))
                
            # Save the evaluated model
            torch.save(net.state_dict(), os.path.join(model_dir, "evaluated_model1.pt"))

    logging.info("Model elapsed:  {:.2f}".format(elapsed))

    return valid_loss / nb_batch, valid_score / nb_batch
#   Save the evaluated model

batch_size = 512

import torch.optim as optim


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net.to(DEVICE)

# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# optimizer = optim.SGD(net.parameters(), lr=0.001)


# Train the model
train_losses=[]
train_scores=[]
valid_losses = []
valid_scores = []

num_epochs = 35
for epoch in range(num_epochs):
    epoch_loss, epoch_score = train_one_epoch(net, batch_size, optimizer, train_loader)
    train_losses.append(epoch_loss)
    train_scores.append(epoch_score)
    valid_loss, valid_score = evaluate(net, batch_size, valid_loader)
    valid_losses.append(valid_loss)
    valid_scores.append(valid_score)
#     print(f"Epoch {epoch+1}: Loss: {epoch_loss}, Score: {epoch_score}")
    print(f"Epoch {epoch+1}: Train Loss: {epoch_loss}, Train Score: {epoch_score}, Valid Loss: {valid_loss}, Valid Score: {valid_score}")

# %% [code] {"jupyter":{"outputs_hidden":false}}
# Plot train loss and valid loss in one plot

# Save the figures in the "plots" folder
plots_dir = "/kaggle/working/plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Save the Train Loss vs. Valid Loss plot
loss_plot_filename = os.path.join(plots_dir, "train_valid_loss_plot.png")

#Plot loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
plt.plot(range(1, num_epochs + 1), valid_losses, label='Valid Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss vs. Valid Loss')
plt.legend()
plt.grid()
plt.savefig(loss_plot_filename)
plt.show()


# Save the Train Accuracy vs. Valid Accuracy plot
accuracy_plot_filename = os.path.join(plots_dir, "train_valid_accuracy_plot.png")
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), train_scores, label='Train Accuracy', marker='o')
plt.plot(range(1, num_epochs + 1), valid_scores, label='Valid Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train Accuracy vs. Valid Accuracy')
plt.legend()
plt.grid()
plt.savefig(accuracy_plot_filename)
# Close the figure to free up memory
plt.show()



In [ ]:
########################################
#    Query ball based algorithm      #
#######################################

import time as ti
start_time = ti.time()
from scipy.sparse import csr_matrix
file_path1 = '/kaggle/input/new-set/clean_data_1000_low_new_sec.csv'
data = pd.read_csv(file_path1, delimiter = ',')

model_path = "/kaggle/input/new-set/trained_model1.pt"
# model_gnn = '/kaggle/input/model-gnn/gnn_model (1).pth'
###   embedded space creation   ####
chunk_size = 1000  
data_chunks = pd.read_csv(file_path1, chunksize=chunk_size, delimiter = ',')#, delimiter = '\t'

# Initialize an empty list to store chunks after processing
processed_chunks = []

# Process each chunk and assign unique HitID

for chunk in data_chunks:
    # Filter rows with muonID not equal to -999
#     chunk = chunk[chunk['muonID'] != -999]

    if not chunk.empty:
        # Generate unique HitID based on index
        chunk['HitID'] = chunk.index + 1

        # Append processed chunk to the list
        processed_chunks.append(chunk)

data = pd.concat(processed_chunks, ignore_index=True)

data.to_csv("/kaggle/working/processed_data_with_hitid1.csv", index=False)

print(data.head())

####  All the classes and definitions  ####

class MLP(nn.Module):
    def __init__(self,
                 nb_hidden,
                 nb_layer,
                 input_dim,
                 mean,
                 std,
                 emb_dim=4):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, nb_hidden)]
        ln = [nn.Linear(nb_hidden, nb_hidden) for _ in range(nb_layer-1)]
        layers.extend(ln)
        self.layers = nn.ModuleList(layers)
        self.emb_layer = nn.Linear(nb_hidden, emb_dim)
        self.act = nn.ReLU()
        self.mean = torch.FloatTensor(mean).to(torch.float)
        self.std = torch.FloatTensor(std).to(torch.float)

    def forward(self, hits):
        hits = self.normalize(hits)
        for l in self.layers:
            hits = l(hits)
            hits = self.act(hits)
            # hits = self.dropout(hits)
        hits = self.emb_layer(hits)
        return hits

#     def normalize(self, hits):

#         return hits
    def normalize(self, hits):

        return hits



#score calculation: correctly predicted labels/total labels

def score_dist_accuracy(pred, true):
    pred_classes = torch.ones_like(pred)  # Initialize all predictions as 1
    pred_classes[pred >= 0.1] = -1 
    correct = pred_classes == true
    nb_correct = correct.sum().item()
    nb_total = true.size(0)
    score = nb_correct / nb_total
    return score

####################
#     PRINTING     #
####################
def print_header():
  '''
  Print header before train / evaluation run.
  '''
  logging.info("         Loss  Score")

def print_eval_stats(nb_processed, loss, score):
  '''
  Log stats during train, evaluation run.
  '''
  logging.info("  {:5d}: {:.3f}  {:2.2f}".format(nb_processed, loss, score))


#defining all the necessary functions

def track_epoch_stats(epoch_nb, lrate, train_stats, val_stats, experiment_dir):
    print("Epoch: {}".format(epoch_nb))
    print("Learning rate: {:.3g}".format(lrate))
    print("Train loss: {:.3f}".format(train_stats[0]))
    print("Train score: {:.2f}".format(train_stats[1]))
    print("Validation loss: {:.3f}".format(val_stats[0]))
    print("Validation score: {:.2f}".format(val_stats[1]))
    print()
def save_test_stats(experiment_dir, test_stats):
    stats = {'loss': test_stats[0],
             'dist_accuracy': test_stats[1]}
    stats_file = os.path.join(experiment_dir, TEST_STATS_FILE)
    with open(stats_file, 'w') as f:
        yaml.dump(stats, f, default_flow_style=False)
        
def print_header():
    '''
    Print header before train / evaluation run.
    '''
    print("         Loss  Score")

def print_eval_stats(nb_processed, loss, score):
    '''
    Log stats during train, evaluation run.
    '''
    print("  {:5d}: {:.3f}  {:2.2f}".format(nb_processed, loss, score))


# def main(args, force=False):

#   experiment_dir = os.path.join(args.artifact_storage_path, 'metric_learning_emb')
  
#   load_path = os.path.join(args.data_storage_path, 'metric_stage_1')
    
#   # Maybe return previously trained model
#   best_net_name = os.path.join(experiment_dir, 'best_model.pkl')
#   if os.path.isfile(best_net_name) and (not force):
#     net = load_model(best_net_name)
#     if not force:
#       print("Best model loaded from previous run. Not forcing training.")
#       return net

#   utils.initialize_experiment_if_needed(experiment_dir, evaluate_only=False)

#   train_path = os.path.join(load_path, 'train.pickle')
#   valid_path = os.path.join(load_path, 'valid.pickle')
#   test_path  = os.path.join(load_path, 'test.pickle')
#   stats_path = os.path.join(load_path, 'stats.yml')

#   train_data = Hit_Pair_Dataset(train_path, 10**8)
#   valid_data = Hit_Pair_Dataset(valid_path, 10**8)
#   test_data  = Hit_Pair_Dataset(test_path, 10**8)


#   train_dataloader = DataLoader(train_dataset,
#                                 batch_size=args.batch_size,
#                                 shuffle=True,
#                                 drop_last=True,
#                                 pin_memory=True,
#                                 num_workers=8)
#   valid_dataloader = DataLoader(valid_dataset,
#                                 batch_size=args.batch_size,
#                                 drop_last=True,
#                                 pin_memory=True,
#                                 num_workers=8)
#   test_dataloader  = DataLoader(test_dataset,
#                                 batch_size=args.batch_size,
#                                 drop_last=True,
#                                 pin_memory=True,
#                                 num_workers=8)

#   net = create_or_restore_model(
#                                     experiment_dir, 
#                                     train_dataset.get_dim(),
#                                     args.nb_hidden, 
#                                     args.nb_layer,
#                                     args.emb_dim,
#                                     stats_path
#                                     )
#   net.to(DEVICE)
#   if next(net.parameters()).is_cuda:
#     logging.warning("Working on GPU")
#     logging.info("GPU type:\n{}".format(torch.cuda.get_device_name(0)))
#   else:
#     logging.warning("Working on CPU")
    
#   train(net,
#         args.lr_start,
#         args.batch_size,
#         args.max_nb_epochs,
#         experiment_dir,
#         train_dataloader,
#         valid_dataloader)

#   # Perform evaluation over test set
#   try:
#     net = load_best_model(experiment_dir).to(DEVICE)
#     logging.warning("\nBest model loaded for evaluation on test set.")
#   except:
#     logging.warning("\nCould not load best model for test set. Using current.")
#   test_stats = evaluate(net, experiment_dir, args.batch_size, test_dataloader, TEST_NAME)
#   utils.save_test_stats(experiment_dir, test_stats)
#   logging.info("Test score:  {:3.2f}".format(test_stats[1]))

#   return net

# input_dim = 4
# nb_hidden = 128
# nb_layer = 3
# emb_dim = 4
# mean = [0, 0, 0, 0]
# std = [1, 1, 1, 1]
# net = MLP(nb_hidden, nb_layer, input_dim, mean, std, emb_dim)

# net.to(DEVICE)

# net.load_state_dict(torch.load(model_path, map_location=DEVICE))

# Now the model should be successfully loaded with the correct state dictionary

input_dim = 4
nb_hidden = 64
nb_layer = 3
emb_dim = 4
mean = [0, 0, 0, 0]
std = [1, 1, 1, 1]
net = MLP(nb_hidden, nb_layer, input_dim, mean, std, emb_dim)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(DEVICE)

# Assuming `raw_data1` is your preprocessed raw dataset (a pandas DataFrame)
raw_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')

# Drop the 'eventID' and 'muonID' columns if they exist
# columns_to_drop = ['eventID', 'muonID']
# raw_data = raw_data1.drop(columns=columns_to_drop, errors='ignore')

# Convert the DataFrame to a PyTorch tensor
# raw_tensor = torch.tensor(raw_data.values, dtype=torch.float32).to(DEVICE)  # Move tensor to the appropriate device
raw_tensor = torch.tensor(raw_data[['pos_x', 'pos_y', 'pos_z', 'time_ns']].values, dtype=torch.float32).to(DEVICE)

# Create a PyTorch Dataset from the tensor
dataset = TensorDataset(raw_tensor)

# Create a DataLoader for the raw dataset
batch_size = 256  # Adjust batch size based on your dataset and computational resources
raw_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
net.eval()

raw_tensor = torch.tensor(processed_data[['pos_x', 'pos_y', 'pos_z', 'time_ns']].values, dtype=torch.float32)
muon_ids = processed_data['track_no'].values  # Extract muonIDs
Hit_ids = processed_data['HitID'].values  # Extract HitIDs

embeddings = []  # List to store the embeddings
distances = []  # List to store the distances

with torch.no_grad():
    for batch_data in raw_loader:
        batch_data = batch_data[0].to(DEVICE)  
        # Calculate embeddings
        emb_points = net(batch_data)
        # Append embeddings to list
        embeddings.append(emb_points.cpu().numpy())
        # Calculate pairwise distances
        pairwise_dist = torch.cdist(emb_points, emb_points, p=2)
        # Append distances to list
        distances.append(pairwise_dist.cpu().numpy())

all_embeddings = np.concatenate(embeddings, axis=0)

max_size = max(arr.shape[0] for arr in distances)

padded_distances = [np.pad(arr, ((0, max_size - arr.shape[0]), (0, max_size - arr.shape[1])), mode='constant') for arr in distances]

all_distances = np.concatenate(padded_distances, axis=1)  # Concatenate along axis 1

np.save('/kaggle/working/all_embeddings1.npy', all_embeddings)
np.save('/kaggle/working/all_distances1.npy', all_distances)

np.save('/kaggle/working/muon_ids1.npy', muon_ids)
np.save('/kaggle/working/Hit_ids1.npy', Hit_ids)

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the necessary data
all_embeddings = np.load('/kaggle/working/all_embeddings1.npy')
muon_ids = np.load('/kaggle/working/muon_ids1.npy')  # Load track_no values

# Generate unique colors for each track_no
unique_track_nos = np.unique(muon_ids)
num_colors = len(unique_track_nos)
palette = sns.color_palette("husl", num_colors)  # Use seaborn for distinct colors
track_no_to_color = {track_no: palette[i] for i, track_no in enumerate(unique_track_nos)}

# Assign colors to each point based on track_no
colors = [track_no_to_color[track_no] for track_no in muon_ids]

# Scatter plot with unique colors for each track
plt.figure(figsize=(8, 6))
plt.scatter(all_embeddings[:, 2], all_embeddings[:, 3], s=10, c=colors, alpha=0.7)
plt.title("Embedded Space with Track Colors")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.show()


######graph and edge refinement ########

embedded_points_table = pd.DataFrame({
    'Index': range(len(all_embeddings)),  # Point indices
    'track_no': muon_ids,  # muonID for each point
    'HitID': Hit_ids,  # HitID for each point
    'Embedded Point': [point.tolist() for point in all_embeddings] 
})

print(embedded_points_table)

embedded_points_table.to_csv('embedded_points_table', index= False)
import os
import pickle
import numpy as np
import pandas as pd
import torch
from sklearn.neighbors import NearestNeighbors
from concurrent.futures import ThreadPoolExecutor
import networkx as nx

# ------------------------------
# Parameters
# ------------------------------
epsilon = 5
batch_size = 100
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')

# Positions and HitIDs
positions = processed_data[['pos_z', 'time_ns']].values
hit_indices = processed_data['HitID'].values

# Nearest Neighbors
neigh = NearestNeighbors(radius=epsilon, algorithm='ball_tree').fit(positions)

# Embeddings
all_embeddings = np.load('/kaggle/working/all_embeddings1.npy')
hit_ids = np.load('/kaggle/working/Hit_ids1.npy')
all_embeddings = torch.tensor(all_embeddings, dtype=torch.float32).cuda()

# Graph directory
if not os.path.exists('graphs1'):
    os.makedirs('graphs1')

# ------------------------------
# Set to Track Processed HitIDs
# ------------------------------
processed_hit_ids = set()

# ------------------------------
# Modified Graph Creation Function
# ------------------------------
def process_batch(batch_query_hit_ids, query_indices):
    """Process a batch of query points into graphs."""
    batch_graphs = []

    query_positions = processed_data.loc[processed_data['HitID'].isin(batch_query_hit_ids), ['pos_z', 'time_ns']].values
    neighbor_indices = neigh.radius_neighbors(query_positions, return_distance=False)

    for query_hit_id, query_idx, neighbors in zip(batch_query_hit_ids, query_indices, neighbor_indices):
        if query_hit_id in processed_hit_ids:
            continue

        # Get full position data from DataFrame
        query_row = processed_data[processed_data['HitID'] == query_hit_id].iloc[0]
        query_z = query_row['pos_z']
        query_time = query_row['time_ns']
        query_point = all_embeddings[query_idx].cpu().numpy()

        # Create graph with full spatial attributes
        G = nx.Graph()
        G.add_node(query_hit_id,
                   features=query_point,
                   pos_x=query_row['pos_x'],  # Added
                   pos_y=query_row['pos_y'],  # Added
                   pos_z=query_z,
                   time=query_time)

        processed_hit_ids.add(query_hit_id)
        valid_neighbors = []

        for neighbor_idx in neighbors:
            if neighbor_idx == query_idx:
                continue

            neighbor_hit_id = hit_indices[neighbor_idx]
            if neighbor_hit_id in processed_hit_ids:
                continue

            # Get neighbor's full position data
            neighbor_row = processed_data[processed_data['HitID'] == neighbor_hit_id].iloc[0]
            neighbor_z = neighbor_row['pos_z']
            neighbor_time = neighbor_row['time_ns']
            neighbor_point = all_embeddings[neighbor_idx].cpu().numpy()

            # Calculate probabilities with full spatial features
            z_diff_sqr = (query_z - neighbor_z) ** 2
            time_diff_sqr = (query_time - neighbor_time) ** 2

            z_prob = 1 / (1 + torch.exp(-10 * torch.tensor(z_diff_sqr, dtype=torch.float32, device='cuda')))
            time_prob = 1 / (1 + torch.exp(-1 * torch.tensor(time_diff_sqr, dtype=torch.float32, device='cuda')))
            prob = z_prob * time_prob

            if prob <= 0.6:
                # Add node with full spatial attributes
                G.add_node(neighbor_hit_id,
                           features=neighbor_point,
                           pos_x=neighbor_row['pos_x'],  # Added
                           pos_y=neighbor_row['pos_y'],  # Added
                           pos_z=neighbor_z,
                           time=neighbor_time)
                G.add_edge(query_hit_id, neighbor_hit_id)
                valid_neighbors.append(neighbor_hit_id)
                processed_hit_ids.add(neighbor_hit_id)

        if len(valid_neighbors) >= 2:
            batch_graphs.append((query_hit_id, G))

    return batch_graphs

# ------------------------------
# Main Execution Loop (Unchanged)
# ------------------------------
num_graphs_saved = 0

with torch.no_grad(), ThreadPoolExecutor() as executor:
    for batch_start_idx in range(0, len(hit_ids), batch_size):
        batch_query_hit_ids = hit_ids[batch_start_idx:batch_start_idx + batch_size]
        batch_query_indices = np.where(np.isin(hit_ids, batch_query_hit_ids))[0]

        batch_graphs = process_batch(batch_query_hit_ids, batch_query_indices)
        num_graphs_saved += len(batch_graphs)

        for graph in batch_graphs:
            query_hit_id, G = graph
            executor.submit(pickle.dump, G, open(f'graphs1/Graph_{query_hit_id}.pkl', 'wb'))

print(f"🔥 Number of graphs saved: {num_graphs_saved}")


import os
import pickle
import networkx as nx
import pandas as pd
import numpy as np

# Load the processed data
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')

# Create a mapping from HitID to true track_no
hitid_to_track = dict(zip(processed_data['HitID'], processed_data['track_no']))

# Count hits per true track
true_track_counts = processed_data['track_no'].value_counts().to_dict()

# Directory containing saved graphs
graph_dir = 'graphs1'

# Load the reconstructed graphs
graphs = []
for file in os.listdir(graph_dir):
    if file.endswith('.pkl'):
        with open(os.path.join(graph_dir, file), 'rb') as f:
            G = pickle.load(f)
            graphs.append(G)

print(f"Number of graphs loaded: {len(graphs)}")

# Extract connected components as reconstructed tracks
reconstructed_tracks = []
for G in graphs:
    for component in nx.connected_components(G):
        reconstructed_tracks.append(list(component))  # Each component is a track (list of HitIDs))

print(f"Total reconstructed tracks: {len(reconstructed_tracks)}")

# Create a dictionary to track how true tracks are reconstructed
true_track_reconstruction = {track_no: [] for track_no in true_track_counts}

# Evaluate track purity and efficiency
valid_tracks = 0
valid_tracks_50 = 0  # For threshold test at 50%

for track in reconstructed_tracks:
    true_track_ids = [hitid_to_track[hit] for hit in track if hit in hitid_to_track]

    if len(true_track_ids) == 0:
        continue

    # Count occurrences of each true track ID in the reconstructed track
    track_counts = np.bincount(true_track_ids)
    dominant_track = np.argmax(track_counts)  # The most frequent true track
    max_hits = track_counts[dominant_track]  # Hits belonging to dominant track
    total_hits = len(track)  # Total hits in this reconstructed track

    purity = max_hits / total_hits

    # Store how many reconstructed fragments exist for each true track
    true_track_reconstruction[dominant_track].append(max_hits)

    # Only consider the reconstructed track if purity is ≥ 70%
    if purity >= 0.7 and max_hits / true_track_counts[dominant_track] >= 0.7:
        valid_tracks += 1
    
    # Also check for a 50% threshold (for debugging)
    if purity >= 0.5 and max_hits / true_track_counts[dominant_track] >= 0.5:
        valid_tracks_50 += 1

# Compute efficiency
total_true_tracks = len(true_track_counts)
efficiency = valid_tracks / total_true_tracks
efficiency_50 = valid_tracks_50 / total_true_tracks  # Lower threshold check

print(f"Corrected Track-Level Efficiency (70% threshold): {efficiency:.3f}")
print(f"Track-Level Efficiency (50% threshold for debugging): {efficiency_50:.3f}")

# Debugging: Check how many true tracks were completely missed
missing_tracks = [track_no for track_no in true_track_counts if not true_track_reconstruction[track_no]]
print(f"Number of true tracks not reconstructed: {len(missing_tracks)}")

# Print sample true track fragmentation details
print("True track fragmentation details:")
for track_no, fragments in list(true_track_reconstruction.items())[:10]:  # Print first 10 for debugging
    print(f"True Track {track_no}: {len(fragments)} reconstructed fragments")

# Debugging: Print a few sample reconstructed tracks
print("Sample reconstructed tracks:")
for i, track in enumerate(reconstructed_tracks[:15]):  # Print first 5 reconstructed tracks
    print(f"Reconstructed Track {i}: {track}")
import os
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the processed data
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')

# Create a mapping from HitID to true track_no (muonID in this case)
hitid_to_track = dict(zip(processed_data['HitID'], processed_data['track_no']))

# Extract true track positions
hit_positions = processed_data.set_index('HitID')[['pos_x', 'pos_y', 'pos_z']]

# Load the reconstructed graphs
graph_dir = 'graphs1'
graphs = []
for file in os.listdir(graph_dir):
    if file.endswith('.pkl'):
        with open(os.path.join(graph_dir, file), 'rb') as f:
            G = pickle.load(f)
            graphs.append(G)

# Extract connected components as reconstructed tracks
reconstructed_tracks = []
for G in graphs:
    for component in nx.connected_components(G):
        reconstructed_tracks.append(list(component))  # Each component is a track (list of HitIDs)

# Select the first 10 reconstructed tracks
num_tracks = 5
selected_reconstructed_tracks = reconstructed_tracks[:num_tracks]

# Find corresponding true tracks
true_tracks = []
for track in selected_reconstructed_tracks:
    # Find the dominant true muonID in this reconstructed track
    true_hit_ids = [hitid_to_track[hit] for hit in track if hit in hitid_to_track]
    if true_hit_ids:
        dominant_muon_id = max(set(true_hit_ids), key=true_hit_ids.count)
        # Get all HitIDs belonging to this true muonID
        true_hit_list = list(processed_data[processed_data['track_no'] == dominant_muon_id]['HitID'])
        true_tracks.append(true_hit_list)
    else:
        true_tracks.append([])  # Empty list if no valid true track found

# Function to plot tracks in 3D
def plot_tracks(tracks, title, color_map='jet'):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    colormap = plt.get_cmap(color_map)
    num_tracks = len(tracks)

    for i, track in enumerate(tracks):
        if not track:
            continue
        valid_hits = [hit for hit in track if hit in hit_positions.index]
        if not valid_hits:
            continue  # Skip empty tracks
        track_hits = hit_positions.loc[valid_hits].values
        ax.scatter(track_hits[:, 0], track_hits[:, 1], track_hits[:, 2], 
                   marker='o', s=10, color=colormap(i / num_tracks), alpha=0.8)

    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_zlabel('Z (mm)')
    ax.set_title(title)
    plt.show()
plot_tracks(selected_reconstructed_tracks, "Reconstructed Tracks (First 10)", color_map='rainbow')
plot_tracks(true_tracks, "True Tracks (Corresponding to First 10 Reconstructed)", color_map='coolwarm')



In [ ]:
import shutil

# Remove all files in the output directory
shutil.rmtree("/kaggle/working/")
os.mkdir("/kaggle/working/")  # Recreate the directory to avoid errors


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

# **🔹 Load Processed Data (for Track Labels)**
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')
hit_to_track = dict(zip(processed_data['HitID'], processed_data['track_no']))

graph_dir = "graphs1/"
graph_purities = []

# **🔹 Iterate Over Saved Graphs**
for filename in os.listdir(graph_dir):
    if filename.endswith(".pkl"):
        with open(os.path.join(graph_dir, filename), "rb") as f:
            G = pickle.load(f)
        
        # **Get Track Labels for Graph Nodes**
        hit_ids = list(G.nodes())
        track_labels = [hit_to_track[hit_id] for hit_id in hit_ids if hit_id in hit_to_track]

        if track_labels:
            # **Compute Track Counts**
            track_counts = Counter(track_labels)
            most_common_track_hits = max(track_counts.values())  # Most frequent track count
            total_hits = len(track_labels)  # Total hits in the graph
            
            purity = most_common_track_hits / total_hits
            graph_purities.append(purity)

# **🔹 Compute Average Purity**
average_purity = np.mean(graph_purities)
print(f"✅ Average Graph Purity: {average_purity:.3f}")

# **🔹 Save Purity Results**
pd.DataFrame({"Graph Purity": graph_purities}).to_csv("graph_purities.csv", index=False)


In [ ]:
file_path1 = '/kaggle/input/new-set/test_1000_low.csv'
data = pd.read_csv(file_path1, delimiter = ',')

model_path = "/kaggle/input/new-set/trained_model1.pt"
model_gnn = '/kaggle/working/edge_classifier.pth'

In [ ]:
# Load the model without applying it
model_dict = torch.load(model_path, map_location=DEVICE)

# Print the keys to inspect the saved layers
print(model_dict.keys())


In [ ]:
import os
import pickle
import networkx as nx
import matplotlib.pyplot as plt

# Directory where graphs are stored
graph_dir = 'graphs2'

# List all graph files
graph_files = [f for f in os.listdir(graph_dir) if f.endswith('.pkl')]

# Select a few graphs to visualize
num_graphs_to_plot = 10  # Change this if needed
selected_graphs = graph_files[:num_graphs_to_plot]  

for graph_file in selected_graphs:
    with open(os.path.join(graph_dir, graph_file), 'rb') as f:
        G = pickle.load(f)

    plt.figure(figsize=(6, 6))
    pos = nx.spring_layout(G)  # Layout for better visualization
    node_colors = [G.nodes[n].get('color', 'gray') for n in G.nodes()]  # Get colors if assigned
    
    nx.draw(G, pos, with_labels=True, node_size=300, node_color=node_colors, edge_color='black')
    plt.title(f"Graph: {graph_file}")
    plt.show()


In [ ]:
import shutil

# Remove all files in the output directory
shutil.rmtree("/kaggle/working/")
os.mkdir("/kaggle/working/")  # Recreate the directory to avoid errors


In [ ]:
import pandas as pd

data_filepath = "/kaggle/input/new-set/pairwise_data_normalized.csv"
dataset = pd.read_csv(data_filepath)
print(dataset.columns)  # Print actual column names


In [ ]:
import trackml

In [ ]:
!pip install trackml


In [ ]:
pip install --user git+https://github.com/LAL/trackml-library

In [ ]:
import shutil

# Remove all files in the output directory
shutil.rmtree("/kaggle/working/")
os.mkdir("/kaggle/working/")  # Recreate the directory to avoid errors


In [ ]:
file_path = '/kaggle/input/new-set/recooutpileup_timemod_uniform_1_100recohitfile.tsv'
data = pd.read_csv(file_path, delimiter = '\t')

model_path = "/kaggle/input/new-set/trained_model1.pt"

model_gnn = '/kaggle/working/track_gnn.pth'
import os
import pickle
import numpy as np
import pandas as pd
import torch
from sklearn.neighbors import NearestNeighbors
from concurrent.futures import ThreadPoolExecutor
import networkx as nx


########################################
#    Query ball based algorithm      #
#######################################

import time as ti
start_time = ti.time()
from scipy.sparse import csr_matrix

###   embedded space creation   ####
chunk_size = 1000  
data_chunks = pd.read_csv(file_path, chunksize=chunk_size, delimiter = '\t')#, delimiter = '\t'
# Initialize an empty list to store chunks after processing
processed_chunks = []
# Process each chunk and assign unique HitID
for chunk in data_chunks:    
     chunk = chunk[chunk['muonID'] != -999]

     if not chunk.empty:
        # Generate unique HitID based on index
        chunk['HitID'] = chunk.index + 1

        # Append processed chunk to the list
        processed_chunks.append(chunk)
data = pd.concat(processed_chunks, ignore_index=True)
data.to_csv("/kaggle/working/processed_data_with_hitid.csv", index=False)
print(data.head())
####  All the classes and definitions  ####

class MLP(nn.Module):
    def __init__(self,
                 nb_hidden,
                 nb_layer,
                 input_dim,
                 mean,
                 std,
                 emb_dim=2):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, nb_hidden)]
        ln = [nn.Linear(nb_hidden, nb_hidden) for _ in range(nb_layer-1)]
        layers.extend(ln)
        self.layers = nn.ModuleList(layers)
        self.emb_layer = nn.Linear(nb_hidden, emb_dim)
        self.act = nn.ReLU()
        self.mean = torch.FloatTensor(mean).to(torch.float)
        self.std = torch.FloatTensor(std).to(torch.float)

    def forward(self, hits):
        hits = self.normalize(hits)
        for l in self.layers:
            hits = l(hits)
            hits = self.act(hits)
            # hits = self.dropout(hits)
        hits = self.emb_layer(hits)
        return hits

#     def normalize(self, hits):

#         return hits
    def normalize(self, hits):
        min_vals = torch.min(hits, dim=1, keepdim=True)[0]
        max_vals = torch.max(hits, dim=1, keepdim=True)[0]
        hits = (hits - min_vals) / (max_vals - min_vals + 1e-10)  # Avoid division by zero
        return hits

#score calculation: correctly predicted labels/total labels

def score_dist_accuracy(pred, true):
    pred_classes = torch.ones_like(pred)  # Initialize all predictions as 1
    pred_classes[pred >= 0.1] = -1 
    correct = pred_classes == true
    nb_correct = correct.sum().item()
    nb_total = true.size(0)
    score = nb_correct / nb_total
    return score

####################
#     PRINTING     #
####################
def print_header():
  '''
  Print header before train / evaluation run.
  '''
  logging.info("         Loss  Score")

def print_eval_stats(nb_processed, loss, score):
  '''
  Log stats during train, evaluation run.
  '''
  logging.info("  {:5d}: {:.3f}  {:2.2f}".format(nb_processed, loss, score))

input_dim = 4
nb_hidden = 64
nb_layer = 3
emb_dim = 4
mean = [0, 0, 0, 0]
std = [1, 1, 1, 1]
net = MLP(nb_hidden, nb_layer, input_dim, mean, std, emb_dim)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(DEVICE)

# Assuming `raw_data1` is your preprocessed raw dataset (a pandas DataFrame)
raw_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')

# Move tensor to the appropriate device
raw_tensor = torch.tensor(raw_data[['pos_x', 'pos_y', 'pos_z', 'time']].values, dtype=torch.float32).to(DEVICE)
# Create a PyTorch Dataset from the tensor
dataset = TensorDataset(raw_tensor)
# Create a DataLoader for the raw dataset
batch_size = 256  # Adjust batch size based on your dataset and computational resources
raw_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
# Set the model to evaluation mode
net.eval()

raw_tensor = torch.tensor(processed_data[['pos_x', 'pos_y', 'pos_z', 'time']].values, dtype=torch.float32)
muon_ids = processed_data['muonID'].values  # Extract muonIDs
Hit_ids = processed_data['HitID'].values  # Extract HitIDs

embeddings = []  # List to store the embeddings
distances = []  # List to store the distances

with torch.no_grad():
    for batch_data in raw_loader:
        batch_data = batch_data[0].to(DEVICE)  
        # Calculate embeddings
        emb_points = net(batch_data)
        # Append embeddings to list
        embeddings.append(emb_points.cpu().numpy())
        # Calculate pairwise distances
        pairwise_dist = torch.cdist(emb_points, emb_points, p=2)
        # Append distances to list
        distances.append(pairwise_dist.cpu().numpy())

all_embeddings = np.concatenate(embeddings, axis=0)
max_size = max(arr.shape[0] for arr in distances)
padded_distances = [np.pad(arr, ((0, max_size - arr.shape[0]), (0, max_size - arr.shape[1])), mode='constant') for arr in distances]
all_distances = np.concatenate(padded_distances, axis=1)  # Concatenate along axis 1
np.save('/kaggle/working/all_embeddings.npy', all_embeddings)
np.save('/kaggle/working/all_distances.npy', all_distances)
np.save('/kaggle/working/muon_ids.npy', muon_ids)
np.save('/kaggle/working/Hit_ids.npy', Hit_ids)
# ------------------------------
# Parameters
# ------------------------------
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import torch
from concurrent.futures import ThreadPoolExecutor

# ------------------------------
# Configuration
# ------------------------------
epsilon = 6
batch_size = 100
graph_dir = 'graphs1'

# ------------------------------
# Data Loading
# ------------------------------
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
positions = processed_data[['pos_z', 'time']].values
hit_indices = processed_data['HitID'].values

neigh = NearestNeighbors(radius=epsilon, algorithm='ball_tree').fit(positions)

all_embeddings = np.load('/kaggle/working/all_embeddings.npy')
hit_ids = np.load('/kaggle/working/Hit_ids.npy')
all_embeddings = torch.tensor(all_embeddings, dtype=torch.float32).cuda()

if not os.path.exists(graph_dir):
    os.makedirs(graph_dir)

# ------------------------------
# Processed HitIDs Tracker
# ------------------------------
processed_hit_ids = set()

# ------------------------------
# Modified Graph Creation Function
# ------------------------------
def process_batch(batch_query_hit_ids, query_indices):
    """Process a batch of query points into graphs."""
    batch_graphs = []

    query_positions = processed_data.loc[processed_data['HitID'].isin(batch_query_hit_ids), ['pos_z', 'time']].values
    neighbor_indices = neigh.radius_neighbors(query_positions, return_distance=False)

    for query_hit_id, query_idx, neighbors in zip(batch_query_hit_ids, query_indices, neighbor_indices):
        if query_hit_id in processed_hit_ids:
            continue

        # Get full position data from DataFrame
        query_row = processed_data[processed_data['HitID'] == query_hit_id].iloc[0]
        query_z = query_row['pos_z']
        query_time = query_row['time']
        query_point = all_embeddings[query_idx].cpu().numpy()

        # Create graph with full spatial attributes
        G = nx.Graph()
        G.add_node(query_hit_id,
                   features=query_point,
                   pos_x=query_row['pos_x'],  # Added
                   pos_y=query_row['pos_y'],  # Added
                   pos_z=query_z,
                   time=query_time)

        processed_hit_ids.add(query_hit_id)
        valid_neighbors = []

        for neighbor_idx in neighbors:
            if neighbor_idx == query_idx:
                continue

            neighbor_hit_id = hit_indices[neighbor_idx]
            if neighbor_hit_id in processed_hit_ids:
                continue

            # Get neighbor's full position data
            neighbor_row = processed_data[processed_data['HitID'] == neighbor_hit_id].iloc[0]
            neighbor_z = neighbor_row['pos_z']
            neighbor_time = neighbor_row['time']
            neighbor_point = all_embeddings[neighbor_idx].cpu().numpy()

            # Calculate probabilities with full spatial features
            z_diff_sqr = (query_z - neighbor_z) ** 2
            time_diff_sqr = (query_time - neighbor_time) ** 2

            z_prob = 1 / (1 + torch.exp(-10 * torch.tensor(z_diff_sqr, dtype=torch.float32, device='cuda')))
            time_prob = 1 / (1 + torch.exp(-1 * torch.tensor(time_diff_sqr, dtype=torch.float32, device='cuda')))
            prob = z_prob * time_prob

            if prob <= 0.6:
                # Add node with full spatial attributes
                G.add_node(neighbor_hit_id,
                           features=neighbor_point,
                           pos_x=neighbor_row['pos_x'],  # Added
                           pos_y=neighbor_row['pos_y'],  # Added
                           pos_z=neighbor_z,
                           time=neighbor_time)
                G.add_edge(query_hit_id, neighbor_hit_id)
                valid_neighbors.append(neighbor_hit_id)
                processed_hit_ids.add(neighbor_hit_id)

        if len(valid_neighbors) >= 2:
            batch_graphs.append((query_hit_id, G))

    return batch_graphs


# ------------------------------
# Main Execution Loop (Unchanged)
# ------------------------------
num_graphs_saved = 0

with torch.no_grad(), ThreadPoolExecutor() as executor:
    for batch_start_idx in range(0, len(hit_ids), batch_size):
        batch_query_hit_ids = hit_ids[batch_start_idx:batch_start_idx + batch_size]
        batch_query_indices = np.where(np.isin(hit_ids, batch_query_hit_ids))[0]

        batch_graphs = process_batch(batch_query_hit_ids, batch_query_indices)
        num_graphs_saved += len(batch_graphs)

        for graph in batch_graphs:
            query_hit_id, G = graph
            executor.submit(pickle.dump, G, open(f'graphs1/Graph_{query_hit_id}.pkl', 'wb'))

print(f"🔥 Number of graphs saved: {num_graphs_saved}")

In [ ]:
import os
import pickle
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load processed data for track_no lookup
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')

# Directory where graphs are saved
graph_dir = 'graphs1'

# List all raw graph files
raw_graph_files = [f for f in os.listdir(graph_dir) if f.startswith('raw_Graph_') and f.endswith('.pkl')]

# How many graphs to plot
import random

# Shuffle the graph files
random.shuffle(raw_graph_files)

# How many graphs to plot
num_graphs_to_plot = 5
selected_graphs = raw_graph_files[:num_graphs_to_plot]


for raw_graph_file in selected_graphs:
    query_hit_id = int(raw_graph_file.split('_')[-1].split('.')[0])  # Extract HitID from filename

    raw_path = os.path.join(graph_dir, f'raw_Graph_{query_hit_id}.pkl')
    filt_path = os.path.join(graph_dir, f'filtered_Graph_{query_hit_id}.pkl')

    if not os.path.exists(filt_path):
        print(f"❌ Missing filtered graph for HitID: {query_hit_id}")
        continue

    # Load graphs
    with open(raw_path, 'rb') as f:
        G_raw = pickle.load(f)
    with open(filt_path, 'rb') as f:
        G_filt = pickle.load(f)

    # Get track number for the query
    try:
        query_track = processed_data.loc[processed_data['HitID'] == query_hit_id, 'muonID'].values[0]
    except IndexError:
        print(f"❌ HitID {query_hit_id} not found in processed data.")
        continue

    # Layout: query in center, neighbors in circle
    neighbors = list(G_raw.neighbors(query_hit_id))
    num_neighbors = max(len(neighbors), 1)
    layout = {query_hit_id: (0, 0)}
    radius = 1.5

    for i, neighbor in enumerate(neighbors):
        angle = 2 * np.pi * i / num_neighbors
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        layout[neighbor] = (x, y)

    # Add extra neighbors from filtered graph
    for node in G_filt.nodes():
        if node not in layout:
            angle = np.random.rand() * 2 * np.pi
            layout[node] = (radius * np.cos(angle), radius * np.sin(angle))

    # Helper function to assign colors
    def assign_colors(G, query_hit_id, query_track):
        colors = []
        for node in G.nodes():
            if node == query_hit_id:
                colors.append('red')
            else:
                try:
                    node_track = processed_data.loc[processed_data['HitID'] == node, 'muonID'].values[0]
                    colors.append('blue' if node_track == query_track else 'green')
                except IndexError:
                    colors.append('gray')
        return colors

    colors_raw = assign_colors(G_raw, query_hit_id, query_track)
    colors_filt = assign_colors(G_filt, query_hit_id, query_track)

    # Plot both graphs side-by-side
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    nx.draw(G_raw, layout, ax=axs[0], with_labels=True, node_size=500, node_color=colors_raw, edge_color='black')
    axs[0].set_title("Original Graph")

    nx.draw(G_filt, layout, ax=axs[1], with_labels=True, node_size=500, node_color=colors_filt, edge_color='black')
    axs[1].set_title("Filtered Graph (After Sigmoid)")

    plt.suptitle(f"Query HitID: {query_hit_id} | Track No: {query_track}", fontsize=14)
    plt.tight_layout()
    plt.show()


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

# 🔹 Load Processed Data (for track labels)
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
hit_to_track = dict(zip(processed_data['HitID'], processed_data['muonID']))

# 🔹 Directory with saved graphs
graph_dir = "/kaggle/working/graphs1"

raw_purities = []
filtered_purities = []

# 🔹 Iterate over filtered graphs (and find matching raw)
for filename in os.listdir(graph_dir):
    if filename.startswith("filtered_Graph_") and filename.endswith(".pkl"):
        query_hit_id = filename.split("filtered_Graph_")[1].split(".pkl")[0]
        raw_path = os.path.join(graph_dir, f"raw_Graph_{query_hit_id}.pkl")
        filtered_path = os.path.join(graph_dir, filename)

        if not os.path.exists(raw_path):
            continue  # Skip if raw graph is missing

        # Load raw and filtered graphs
        with open(raw_path, "rb") as f:
            raw_G = pickle.load(f)
        with open(filtered_path, "rb") as f:
            filtered_G = pickle.load(f)

        def compute_purity(G):
            hit_ids = list(G.nodes())
            track_labels = [hit_to_track[hit_id] for hit_id in hit_ids if hit_id in hit_to_track]
            if not track_labels:
                return None
            track_counts = Counter(track_labels)
            most_common_track_hits = max(track_counts.values())
            return most_common_track_hits / len(track_labels)

        raw_purity = compute_purity(raw_G)
        filtered_purity = compute_purity(filtered_G)

        if raw_purity is not None and filtered_purity is not None:
            raw_purities.append(raw_purity)
            filtered_purities.append(filtered_purity)

# 🔹 Report average purities
raw_avg = np.mean(raw_purities) * 100
filtered_avg = np.mean(filtered_purities) * 100

print(f"📊 Average Raw Graph Purity     : {raw_avg:.2f}%")
print(f"✅ Average Filtered Graph Purity: {filtered_avg:.2f}%")



In [ ]:
file_path = '/kaggle/input/new-set/recooutpileup_timemod_uniform_1_100recohitfile.tsv'
data = pd.read_csv(file_path, delimiter='\t')

model_path = "/kaggle/input/new-set/trained_model1.pt"
model_gnn = '/kaggle/working/track_gnn.pth'

import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.neighbors import NearestNeighbors
from concurrent.futures import ThreadPoolExecutor
import networkx as nx
import time as ti

start_time = ti.time()

# ------------------------------
# Data Processing
# ------------------------------
chunk_size = 1000  
data_chunks = pd.read_csv(file_path, chunksize=chunk_size, delimiter='\t')
processed_chunks = []

for chunk in data_chunks:    
    chunk = chunk[chunk['muonID'] != -999]
    if not chunk.empty:
        chunk['HitID'] = chunk.index + 1
        processed_chunks.append(chunk)

data = pd.concat(processed_chunks, ignore_index=True)
data.to_csv("/kaggle/working/processed_data_with_hitid.csv", index=False)
print("Processed data sample:\n", data.head())

# ------------------------------
# MLP Model Definition
# ------------------------------
class MLP(nn.Module):
    def __init__(self, nb_hidden, nb_layer, input_dim, mean, std, emb_dim=4):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, nb_hidden)]
        layers.extend([nn.Linear(nb_hidden, nb_hidden) for _ in range(nb_layer-1)])
        self.layers = nn.ModuleList(layers)
        self.emb_layer = nn.Linear(nb_hidden, emb_dim)
        self.act = nn.ReLU()
        self.mean = torch.FloatTensor(mean)
        self.std = torch.FloatTensor(std)

    def normalize(self, hits):
        min_vals = torch.min(hits, dim=1, keepdim=True)[0]
        max_vals = torch.max(hits, dim=1, keepdim=True)[0]
        return (hits - min_vals) / (max_vals - min_vals + 1e-10)

    def forward(self, hits):
        hits = self.normalize(hits)
        for layer in self.layers:
            hits = layer(hits)
            hits = self.act(hits)
        return self.emb_layer(hits)

# ------------------------------
# Device Configuration
# ------------------------------
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# ------------------------------
# Model Initialization
# ------------------------------
input_dim = 4
nb_hidden = 64
nb_layer = 3
emb_dim = 4
mean = [0, 0, 0, 0]
std = [1, 1, 1, 1]
net = MLP(nb_hidden, nb_layer, input_dim, mean, std, emb_dim).to(DEVICE)
net.load_state_dict(torch.load(model_path))
net.eval()

# ------------------------------
# Data Loading & Embedding Generation
# ------------------------------
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')

# Generate embeddings
with torch.no_grad():
    input_tensor = torch.tensor(
        processed_data[['pos_x', 'pos_y', 'pos_z', 'time']].values,
        dtype=torch.float32,
        device=DEVICE
    )
    all_embeddings = net(input_tensor).cpu().numpy()

# Save embeddings and IDs
hit_ids = processed_data['HitID'].values
np.save('/kaggle/working/all_embeddings.npy', all_embeddings)
np.save('/kaggle/working/Hit_ids.npy', hit_ids)

# ------------------------------
# Graph Creation Parameters
# ------------------------------
epsilon = 1  # Increased search radius
prob_threshold = 0.5  # More inclusive threshold
batch_size = 1000  # Larger batch size for efficiency
min_neighbors = 2  # Minimum neighbors required

# ------------------------------
# Neighbor Search Setup
# ------------------------------
positions = processed_data[['pos_z', 'time']].values
neigh = NearestNeighbors(radius=epsilon, algorithm='ball_tree')
neigh.fit(positions)

# ------------------------------
# Graph Creation Function (Fixed)
# ------------------------------
processed_hit_ids = set()

def process_batch(batch_query_hit_ids):
    batch_graphs = []
    query_indices = np.where(np.isin(hit_ids, batch_query_hit_ids))[0]
    
    if len(query_indices) == 0:
        return batch_graphs

    query_positions = positions[query_indices]
    neighbor_indices = neigh.radius_neighbors(query_positions, return_distance=False)

    for q_hit_id, q_idx, neighbors in zip(batch_query_hit_ids, query_indices, neighbor_indices):
        if q_hit_id in processed_hit_ids:
            continue

        # Create graph even if no neighbors
        G = nx.Graph()
        q_data = processed_data[processed_data['HitID'] == q_hit_id].iloc[0]
        G.add_node(q_hit_id,
                   features=all_embeddings[q_idx],
                   pos_z=q_data['pos_z'],
                   time=q_data['time'])

        valid_neighbors = []
        for n_idx in neighbors:
            if n_idx == q_idx:
                continue

            n_hit_id = hit_ids[n_idx]
            if n_hit_id in processed_hit_ids:
                continue

            n_data = processed_data[processed_data['HitID'] == n_hit_id].iloc[0]
            
            # Calculate connection probability
            z_diff = q_data['pos_z'] - n_data['pos_z']
            time_diff = q_data['time'] - n_data['time']
            z_prob = 1 / (1 + np.exp(-10 * z_diff**2))
            time_prob = 1 / (1 + np.exp(-1 * time_diff**2))
            prob = z_prob * time_prob

            if prob >= prob_threshold:
                G.add_node(n_hit_id,
                           features=all_embeddings[n_idx],
                           pos_z=n_data['pos_z'],
                           time=n_data['time'])
                G.add_edge(q_hit_id, n_hit_id)
                valid_neighbors.append(n_hit_id)

        # Always save the graph regardless of neighbor count
        batch_graphs.append((q_hit_id, G))
        processed_hit_ids.add(q_hit_id)
        processed_hit_ids.update(valid_neighbors)

    return batch_graphs

# ------------------------------
# Main Processing Loop
# ------------------------------
if not os.path.exists('graphs'):
    os.makedirs('graphs')

all_hit_ids = processed_data['HitID'].unique()
num_graphs_saved = 0

with ThreadPoolExecutor() as executor:
    for i in range(0, len(all_hit_ids), batch_size):
        batch_ids = all_hit_ids[i:i + batch_size]
        batch_graphs = process_batch(batch_ids)
        num_graphs_saved += len(batch_graphs)

        # Save graphs in parallel
        futures = []
        for q_hit_id, G in batch_graphs:
            future = executor.submit(
                pickle.dump, 
                G, 
                open(f'graphs/Graph_{q_hit_id}.pkl', 'wb')
            )
            futures.append(future)
        
        # Wait for completion of this batch
        for future in futures:
            future.result()

print(f"🔥 Total graphs saved: {num_graphs_saved}")

# ------------------------------
# Post-Processing Validation
# ------------------------------
all_processed = set(processed_hit_ids)
missing_hits = set(all_hit_ids) - all_processed

print(f"\nValidation Results:")
print(f"Original HitIDs: {len(all_hit_ids)}")
print(f"Processed HitIDs: {len(all_processed)}")
print(f"Missing HitIDs: {len(missing_hits)} ({len(missing_hits)/len(all_hit_ids):.2%})")




print(f"\nTotal execution time: {ti.time() - start_time:.2f} seconds")

In [ ]:
import pandas as pd
import pickle
import os
import networkx as nx

# Load original HitIDs
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')
original_hit_ids = set(processed_data['HitID'].values)

# Load HitIDs from saved graphs
graph_hit_ids = set()
graph_dir = 'graphs1'

for filename in os.listdir(graph_dir):
    if filename.endswith('.pkl'):
        with open(os.path.join(graph_dir, filename), 'rb') as f:
            G = pickle.load(f)  # Load graph
            graph_hit_ids.update(G.nodes)  # Collect all HitIDs in the graph

# Compare counts
print(f"Original number of HitIDs: {len(original_hit_ids)}")
print(f"Number of HitIDs after graph creation: {len(graph_hit_ids)}")

# Check if any HitIDs are missing
missing_hit_ids = original_hit_ids - graph_hit_ids
extra_hit_ids = graph_hit_ids - original_hit_ids

if missing_hit_ids:
    print(f"⚠️ {len(missing_hit_ids)} HitIDs missing after graph creation!")
    print(f"Examples of missing HitIDs: {list(missing_hit_ids)[:10]}")
else:
    print("✅ No HitIDs were lost during graph creation.")

if extra_hit_ids:
    print(f"⚠️ {len(extra_hit_ids)} unexpected HitIDs found in the graphs!")
    print(f"Examples of extra HitIDs: {list(extra_hit_ids)[:10]}")
else:
    print("✅ No unexpected HitIDs were added.")


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

# **🔹 Load Processed Data (for Track Labels)**
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')
hit_to_track = dict(zip(processed_data['HitID'], processed_data['track_no']))

graph_dir = "graphs1/"
graph_purities = []

# **🔹 Iterate Over Saved Graphs**
for filename in os.listdir(graph_dir):
    if filename.endswith(".pkl"):
        with open(os.path.join(graph_dir, filename), "rb") as f:
            G = pickle.load(f)
        
        # **Get Track Labels for Graph Nodes**
        hit_ids = list(G.nodes())
        track_labels = [hit_to_track[hit_id] for hit_id in hit_ids if hit_id in hit_to_track]

        if track_labels:
            # **Compute Track Counts**
            track_counts = Counter(track_labels)
            most_common_track_hits = max(track_counts.values())  # Most frequent track count
            total_hits = len(track_labels)  # Total hits in the graph
            
            purity = most_common_track_hits / total_hits
            graph_purities.append(purity)

# **🔹 Compute Average Purity**
average_purity = (np.mean(graph_purities))*100
print(f"✅ Average Graph Purity: {average_purity:.3f}%")

# **🔹 Save Purity Results**
pd.DataFrame({"Graph Purity": graph_purities}).to_csv("graph_purities.csv", index=False)


In [ ]:
# Check for duplicate graphs
graph_ids = [file.split('_')[1].split('.')[0] for file in os.listdir(graph_dir)]
print(f"Number of unique graphs: {len(set(graph_ids))}")
print(f"Total graphs saved: {len(graph_ids)}")


In [ ]:
import pickle
import os

# Load one sample graph
graph_path = '/kaggle/working/graphs'  # Replace with your actual graph directory
sample_graph_file = next((f for f in os.listdir(graph_path) if f.endswith('.pkl')), None)

if sample_graph_file:
    sample_graph = pickle.load(open(os.path.join(graph_path, sample_graph_file), 'rb'))

    # Inspect the graph properties
    print(f"Graph: {sample_graph_file}")
    print("Nodes and their attributes:")

    # Print the first few nodes and their attributes
    for node, data in list(sample_graph.nodes(data=True))[:5]:
        print(f"Node {node}: {data}")

    # Check edges
    print("\nSample Edges:")
    print(list(sample_graph.edges(data=True))[:5])
else:
    print("No graph files found.")


In [ ]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset, DataLoader, Batch
from torch_geometric.nn import GCNConv, global_mean_pool
import itertools
from sklearn.utils import shuffle 

# ------------------------------
# Enhanced Configuration
# ------------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processed_data_path = '/kaggle/working/processed_data_with_hitid.csv'
graph_dir = 'graphs'
model_dir = 'saved_models'
checkpoint_dir = os.path.join(model_dir, 'checkpoints')

os.makedirs(model_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Enhanced training parameters
test_size = 0.2
random_state = 42
epochs = 30
checkpoint_freq = 5
batch_size = 128
max_pairs_per_track = 20

# Regularization parameters
dropout_rate = 0.5
weight_decay = 1e-4
max_grad_norm = 1.0

# ------------------------------
# Data Loading & Preparation
# ------------------------------
def load_graph_metadata():
    print("⏳ Loading graph metadata...")
    processed_data = pd.read_csv(processed_data_path)
    hitid_to_track = dict(zip(processed_data['HitID'], processed_data['muonID']))
    
    metadata = []
    graph_files = [f for f in os.listdir(graph_dir) if f.endswith('.pkl')]
    
    for gf in tqdm(graph_files, desc="Processing graphs"):
        hit_id = int(gf.split('_')[1].split('.')[0])
        metadata.append({
            'file_path': os.path.join(graph_dir, gf),
            'hit_id': hit_id,
            'track_id': hitid_to_track[hit_id]
        })
    
    return pd.DataFrame(metadata)

# ------------------------------
# Enhanced Pair Generation
# ------------------------------
def generate_pairs(metadata):
    print("\n🎯 Generating balanced pairs...")
    positive_pairs = []
    negative_pairs = []
    negative_types = []  # Track negative pair types
    
    track_groups = metadata.groupby('track_id')
    track_dict = {track: group['file_path'].tolist() 
                 for track, group in track_groups}
    
    # Generate positive pairs
    for track, graphs in tqdm(track_dict.items(), desc="Positive pairs"):
        if len(graphs) >= 2:
            possible_pairs = list(itertools.combinations(graphs, 2))
            n_pairs = min(len(possible_pairs), max_pairs_per_track)
            positive_pairs.extend(random.sample(possible_pairs, n_pairs))
    
    num_positive = len(positive_pairs)
    print(f"✅ Generated {num_positive} positive pairs")
    
    # Track features for hard negative mining
    track_features = {}
    for track, paths in track_dict.items():
        with open(paths[0], 'rb') as f:
            G = pickle.load(f)
        track_features[track] = list(G.nodes(data=True))[0][1]['features']
    
    # Generate negatives with type tracking
    tracks = list(track_dict.keys())
    existing_pairs = set()
    hard_neg_ratio = 0.9
    
    with tqdm(total=num_positive, desc="Generating negatives") as pbar:
        while len(negative_pairs) < num_positive:
            if random.random() < hard_neg_ratio:
                # Hard negative generation
                track1 = random.choice(tracks)
                similarities = [
                    (np.dot(track_features[track1], track_features[t2]), t2)
                    for t2 in random.sample(tracks, min(50, len(tracks)))
                    if t2 != track1
                ]
                if similarities:
                    track2 = max(similarities)[1]
                    neg_type = 'hard'
                else:
                    continue
            else:
                # Random negative
                track1, track2 = random.sample(tracks, 2)
                neg_type = 'random'
            
            try:
                g1 = random.choice(track_dict[track1])
                g2 = random.choice(track_dict[track2])
            except (KeyError, IndexError):
                continue
            
            pair = frozenset({g1, g2})
            if pair not in existing_pairs:
                negative_pairs.append((g1, g2))
                negative_types.append(neg_type)
                existing_pairs.add(pair)
                pbar.update(1)
    
    print(f"✅ Generated {len(negative_pairs)} negative pairs "
          f"({sum(t == 'hard' for t in negative_types)} hard, "
          f"{sum(t == 'random' for t in negative_types)} random)")
    
    # Create stratified labels
    strat_labels = ['positive'] * num_positive + negative_types
    all_pairs = positive_pairs + negative_pairs
    all_labels = [1] * num_positive + [0] * num_positive
    
    # Shuffle maintaining stratification
    all_pairs, all_labels, strat_labels = shuffle(
        all_pairs, all_labels, strat_labels, 
        random_state=42
    )
    
    return all_pairs, all_labels, strat_labels
# ------------------------------
# Enhanced Dataset Class with Validation
# ------------------------------
class GraphPairDataset(Dataset):
    def __init__(self, pairs, labels):
        super().__init__()
        self.pairs = pairs
        self.labels = labels
        self.cache = {}
        
    def validate_graph(self, data):
        """Edge index validation"""
        if data.edge_index.max() >= data.num_nodes:
            raise ValueError(f"Edge index exceeds node count (max {data.num_nodes-1})")
        if data.edge_index.min() < 0:
            raise ValueError("Negative edge indices detected")
        # Removed duplicate edge check to handle undirected graphs

    def load_graph(self, path):
        if path not in self.cache:
            with open(path, 'rb') as f:
                G = pickle.load(f)
            
            nodes = list(G.nodes())
            node_id_to_idx = {n: i for i, n in enumerate(nodes)}
            
            edge_list = []
            seen_edges = set()  # Prevent duplicate edges
            for u, v in G.edges():
                if u not in node_id_to_idx or v not in node_id_to_idx:
                    raise ValueError(f"Invalid edge ({u}->{v}) in {path}")
                src = node_id_to_idx[u]
                dst = node_id_to_idx[v]
                
                # Check for reverse duplicates
                if (src, dst) in seen_edges or (dst, src) in seen_edges:
                    continue
                seen_edges.add((src, dst))
                
                edge_list.append([src, dst])
            
            data = Data(
                x=torch.tensor([G.nodes[n]['features'] for n in nodes], dtype=torch.float32),
                edge_index=torch.tensor(edge_list).t().contiguous(),
                num_nodes=len(nodes)
            )
            
            self.validate_graph(data)
            self.cache[path] = data
            
        return self.cache[path]
    
    def len(self):
        return len(self.pairs)
    
    def get(self, idx):
        g1_path, g2_path = self.pairs[idx]
        return (
            self.load_graph(g1_path), 
            self.load_graph(g2_path), 
            torch.tensor(self.labels[idx], dtype=torch.float32)
        )
# ------------------------------
# Enhanced GNN with Edge Processing
# ------------------------------
class EnhancedEdgeNetwork(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(2*input_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self.edge_mlp = nn.Sequential(
            nn.Linear(2*input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    
    def forward(self, src, dest):
        attn_weights = self.attention(torch.cat([src, dest], dim=-1))
        return attn_weights * self.edge_mlp(torch.cat([src, dest], dim=-1))

class TrackGNN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=512, embed_dim=256, n_iter=3):
        super().__init__()
        self.n_iter = n_iter
        
        # GCN Layers with increased capacity
        self.convs = nn.ModuleList([
            GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        # Enhanced edge networks
        self.edge_nets = nn.ModuleList([
            EnhancedEdgeNetwork(input_dim if i == 0 else hidden_dim)
            for i in range(n_iter)
        ])
        
        # Residual connections
        self.res_lin = nn.ModuleList([
            nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        # Batch Norms for stability
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(hidden_dim) for _ in range(n_iter)
        ])
        
        # Enhanced embedding projection
        self.lin = nn.Sequential(
            nn.Linear(hidden_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # Scaled-up classifier
        self.classifier = nn.Sequential(
            nn.Linear(2*embed_dim, 512),  # Increased to match new embed_dim
            nn.ReLU(),
            nn.LayerNorm(512),
            nn.Dropout(0.2),
            nn.Linear(512, 256),          # Scaled intermediate layer
            nn.ReLU(),
            nn.LayerNorm(256),
            nn.Linear(256, 1),            # Final projection
            nn.Sigmoid()
        )

    def forward(self, data1, data2):
        # [Unchanged forward implementation]
        def process(data):
            x, edge_index = data.x, data.edge_index
            edge_row, edge_col = edge_index
            
            for i in range(self.n_iter):
                edge_weights = torch.sigmoid(
                    self.edge_nets[i](x[edge_row], x[edge_col])
                ).squeeze()
                
                x_new = F.relu(self.bns[i](self.convs[i](x, edge_index, edge_weights)))
                x = x_new + self.res_lin[i](x)
                
                if self.training:
                    x += torch.randn_like(x) * 0.01
                    
            return global_mean_pool(self.lin(x), data.batch)
        
        emb1 = process(data1)
        emb2 = process(data2)
        return self.classifier(torch.cat([emb1, emb2], dim=-1)).squeeze()

# Rest of the Code Remains Unchanged
# ------------------------------
def collate_fn(batch):
    data1_list, data2_list, label_list = [], [], []
    for data1, data2, label in batch:
        data1_list.append(data1)
        data2_list.append(data2)
        label_list.append(label)
    return (
        Batch.from_data_list(data1_list),
        Batch.from_data_list(data2_list),
        torch.stack(label_list)
    )

def save_model(model, history, suffix=None):
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    model_name = f'track_gnn_{timestamp}' if not suffix else f'track_gnn_{suffix}'
    torch.save({
        'model_state': model.state_dict(),
        'history': history,
        'config': {
            'input_dim': model.convs[0].in_channels,
            'hidden_dim': model.convs[0].out_channels,
            'embed_dim': model.lin[0].out_features  # Access first layer of Sequential
        }
    }, os.path.join(model_dir, f'{model_name}.pt'))
    print(f"💾 Model saved as {model_name}.pt")


def load_model(model_path):
    checkpoint = torch.load(model_path, map_location=device)
    model = TrackGNN(
        input_dim=checkpoint['config']['input_dim'],
        hidden_dim=checkpoint['config']['hidden_dim'],
        embed_dim=checkpoint['config']['embed_dim']
    ).to(device)
    model.load_state_dict(checkpoint['model_state'])
    return model, checkpoint['history']

def plot_history(history):
    plt.figure(figsize=(12, 5))
    
    # Loss Curves
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['test_loss'], label='Test Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')         # Added x-axis label
    plt.ylabel('Loss')           # Added y-axis label
    plt.legend()

    # Accuracy Curves
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['test_acc'], label='Test Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epochs')         # Added x-axis label
    plt.ylabel('Accuracy')       # Added y-axis label
    plt.legend()

    plt.tight_layout()
    plt.show()
def train_model():
    # Load and prepare data
    metadata_df = load_graph_metadata()
    pairs, labels, strat_labels = generate_pairs(metadata_df)
    
    # Enhanced stratified split
    X_train, X_test, y_train, y_test = train_test_split(
        pairs, labels,
        test_size=test_size,
        random_state=random_state,
        stratify=strat_labels  # Stratify by pair type
    )
    
    # Verify stratification
    # Get indices for all pairs
    all_indices = np.arange(len(pairs))
    
    # Get boolean mask for training samples
    in_train = np.zeros(len(pairs), dtype=bool)
    train_indices, _ = train_test_split(
        all_indices,
        test_size=test_size,
        stratify=strat_labels,
        random_state=random_state
    )
    in_train[train_indices] = True
    
    # Get actual stratification labels
    train_strat = [s for s, keep in zip(strat_labels, in_train) if keep]
    test_strat = [s for s, keep in zip(strat_labels, in_train) if not keep]
    
    print("\nStratification Report:")
    print(f"Train: {len(X_train)} pairs "
          f"({train_strat.count('positive')/len(X_train):.1%} pos, "
          f"{train_strat.count('hard')/len(X_train):.1%} hard neg, "
          f"{train_strat.count('random')/len(X_train):.1%} rand neg)")
    print(f"Test:  {len(X_test)} pairs "
          f"({test_strat.count('positive')/len(X_test):.1%} pos, "
          f"{test_strat.count('hard')/len(X_test):.1%} hard neg, "
          f"{test_strat.count('random')/len(X_test):.1%} rand neg)")

    # Initialize model components
    model = TrackGNN().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
    criterion = torch.nn.BCELoss()

    # Create datasets and loaders
    train_dataset = GraphPairDataset(X_train, y_train)
    test_dataset = GraphPairDataset(X_test, y_test)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        collate_fn=collate_fn
    )
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        shuffle=False,
        collate_fn=collate_fn
    )

    # Training history tracking
    history = {'train_loss': [], 'test_loss': [], 'train_acc': [], 'test_acc': []}

    # Training loop
    for epoch in range(epochs):
        print(f"\n🎯 Epoch {epoch+1}/{epochs}")
        model.train()
        train_loss, train_correct = 0, 0
        
        with tqdm(train_loader, unit="batch", desc="Training") as tepoch:
            for data1, data2, labels in tepoch:
                data1, data2 = data1.to(device), data2.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(data1, data2)
                
                assert outputs.shape == labels.shape, \
                    f"Shape mismatch: outputs {outputs.shape}, labels {labels.shape}"
                
                loss = criterion(outputs, labels)
                loss.backward()
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                
                train_loss += loss.item()
                train_correct += ((outputs > 0.5).float() == labels).sum().item()
                tepoch.set_postfix(loss=loss.item())

        model.eval()
        test_loss, test_correct = 0, 0
        with torch.no_grad(), tqdm(test_loader, unit="batch", desc="Testing") as tepoch:
            for data1, data2, labels in tepoch:
                data1, data2 = data1.to(device), data2.to(device)
                labels = labels.to(device)
                
                outputs = model(data1, data2)
                test_loss += criterion(outputs, labels).item()
                test_correct += ((outputs > 0.5).float() == labels).sum().item()
                tepoch.set_postfix(loss=criterion(outputs, labels).item())

        scheduler.step(test_loss/len(test_loader))

        train_loss_avg = train_loss/len(train_loader)
        test_loss_avg = test_loss/len(test_loader)
        train_acc = train_correct/len(X_train)
        test_acc = test_correct/len(X_test)
        
        history['train_loss'].append(train_loss_avg)
        history['test_loss'].append(test_loss_avg)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)

        if (epoch+1) % checkpoint_freq == 0:
            checkpoint_name = f'checkpoint_epoch_{epoch+1}.pt'
            torch.save({
                'model_state': model.state_dict(),
                'history': history,
                'epoch': epoch+1
            }, os.path.join(checkpoint_dir, checkpoint_name))
            print(f"💾 Saved checkpoint at epoch {epoch+1}")

        print(f"Train Loss: {train_loss_avg:.4f} | Test Loss: {test_loss_avg:.4f}")
        print(f"Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.2e}")

    save_model(model, history)
    plot_history(history)
    
    return model, history
if __name__ == "__main__":
    trained_model, training_history = train_model()

In [ ]:
import os
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_mean_pool

from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors
import torch.nn.functional as F


# 1. Define Model Architecture (Must match training exactly)
class EnhancedEdgeNetwork(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 128),
            torch.nn.Tanh(),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, src, dest):
        attn_weights = self.attention(torch.cat([src, dest], dim=-1))
        return attn_weights * self.edge_mlp(torch.cat([src, dest], dim=-1))

class TrackGNN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=512, embed_dim=256, n_iter=3):
        super().__init__()
        self.n_iter = n_iter
        
        # Original encoder components
        self.convs = torch.nn.ModuleList([
            GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.edge_nets = torch.nn.ModuleList([
            EnhancedEdgeNetwork(input_dim if i == 0 else hidden_dim)
            for i in range(n_iter)
        ])
        
        self.res_lin = torch.nn.ModuleList([
            torch.nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(hidden_dim) for _ in range(n_iter)
        ])
        
        self.lin = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, embed_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )

        # Add missing classifier with exact architecture from training
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2*embed_dim, 512),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(512),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(256),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, data):
        # Keep original forward logic
        x, edge_index = data.x, data.edge_index
        edge_row, edge_col = edge_index
        
        for i in range(self.n_iter):
            edge_weights = torch.sigmoid(
                self.edge_nets[i](x[edge_row], x[edge_col])
            ).squeeze()
            
            x_new = F.relu(self.bns[i](self.convs[i](x, edge_index, edge_weights)))
            x = x_new + self.res_lin[i](x)
            
        return global_mean_pool(self.lin(x), data.batch)


# 2. Dataset Class for Inference
from torch_geometric.data import Dataset

class GraphDataset(Dataset):
    def __init__(self, graph_dir):
        super().__init__()
        self.graph_dir = graph_dir
        self.file_list = sorted([f for f in os.listdir(graph_dir) if f.endswith('.pkl')])
        self.hit_ids = [self._extract_hit_id(f) for f in self.file_list]  # Store hit IDs
        self.cache = {}

    def _extract_hit_id(self, filename):
        """Extract HitID from filename (e.g., Graph_12345.pkl -> 12345)"""
        return int(filename.split('_')[1].split('.')[0])

    def load_graph(self, path):
        if path not in self.cache:
            with open(path, 'rb') as f:
                G = pickle.load(f)
            
            nodes = list(G.nodes())
            node_id_to_idx = {n: i for i, n in enumerate(nodes)}
            
            edge_list = []
            seen_edges = set()
            for u, v in G.edges():
                src = node_id_to_idx[u]
                dst = node_id_to_idx[v]
                if (src, dst) not in seen_edges and (dst, src) not in seen_edges:
                    edge_list.append([src, dst])
                    seen_edges.add((src, dst))
            
            self.cache[path] = Data(
                x=torch.tensor([G.nodes[n]['features'] for n in nodes], dtype=torch.float32),
                edge_index=torch.tensor(edge_list).t().contiguous(),
                num_nodes=len(nodes)
            )
        return self.cache[path]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        return self.load_graph(os.path.join(self.graph_dir, self.file_list[idx]))

    def get_hit_ids(self):
        """Get all HitIDs in dataset order"""
        return self.hit_ids




import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
import torch
from torch_geometric.data import Data, DataLoader


def merge_tracks(model_path, test_graph_dir, output_csv):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load original data with validation
    processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid1.csv')
    if 'track_no' not in processed_data.columns:
        raise ValueError("Processed data missing 'muonID' column")
    original_tracks = processed_data['track_no'].nunique()
    
    # Load model with safety checks
    checkpoint = torch.load(model_path, map_location=device)
    model = TrackGNN(
        input_dim=checkpoint['config'].get('input_dim', 4),
        hidden_dim=checkpoint['config'].get('hidden_dim', 512),
        embed_dim=checkpoint['config'].get('embed_dim', 256)
    ).to(device)
    model.load_state_dict(checkpoint['model_state'], strict=False)
    model.eval()

    # Batch processing setup
    dataset = GraphDataset(test_graph_dir)
    hit_ids = dataset.hit_ids
    
    loader = DataLoader(
        dataset,
        batch_size=64,
        num_workers=4,
        pin_memory=True
    )

    # Process graphs in batches
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing graphs"):
            batch = batch.to(device)
            embeddings.append(model(batch).cpu().numpy())
    
    X = np.concatenate(embeddings)
    X = StandardScaler().fit_transform(X)

    # Density-based clustering with updated parameters
    clusterer = DBSCAN(
        eps=0.5,        # Increased epsilon
        min_samples=2,   # Reduced minimum points
        metric='euclidean',
        n_jobs=-1
    ).fit(X)
    
    # Create results without size constraints
    results = pd.DataFrame({
        'HitID': hit_ids,
        'TrackID': clusterer.labels_,
    }).merge(
        processed_data[['HitID', 'track_no']],
        on='HitID',
        how='left'
    )

    # Cluster validation without size limits
    cluster_stats = results[results['TrackID'] != -1].groupby('TrackID').agg(
        size=('HitID', 'count'),
        purity=('track_no', lambda x: x.value_counts(normalize=True).iloc[0])
    ).reset_index()

    # Remove all size constraints
    min_size = 1  # Absolute minimum
    min_purity = 0.65  # Relaxed purity threshold

    # Final validation (only purity check)
    valid_tracks = cluster_stats[
        (cluster_stats['purity'] >= min_purity)
    ]

    # Efficiency calculation
    if valid_tracks.empty:
        efficiency = 0.0
    else:
        efficiency = valid_tracks.merge(
            results[['TrackID', 'track_no']].drop_duplicates(),
            on='TrackID',
            how='left'
        )['track_no'].nunique() / original_tracks

    # Diagnostic reporting
    print("\n=== Clustering Diagnostics ===")
    print(f"Total clusters: {len(cluster_stats)}")
    print(f"Noise points: {(results['TrackID'] == -1).sum()} ({(results['TrackID'] == -1).mean():.1%})")
    print(f"Cluster size range: {cluster_stats['size'].min()}-{cluster_stats['size'].max()}")
    print(f"Validation thresholds: purity ≥ {min_purity:.2f}")
    
    print("\n=== Final Results ===")
    print(f"Original muon tracks: {original_tracks}")
    print(f"Valid reconstructed tracks: {len(valid_tracks)}")
    print(f"Tracking efficiency: {efficiency:.2%}")

    results.to_csv(output_csv, index=False)
    return results, valid_tracks, efficiency, X  # Add X to the return



# Call the function and get results + embeddings
results, stats, efficiency, X = merge_tracks(
    model_path="/kaggle/input/new-set/track_gnn_best.pt",
    test_graph_dir="/kaggle/working/graphs1",
    output_csv="merged_tracks.csv"
)

# Now do the t-SNE visualization
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=2, perplexity=40, random_state=42)
X_2d = tsne.fit_transform(X)

# Merge X_2d with labels
results['x'] = X_2d[:, 0]
results['y'] = X_2d[:, 1]

plt.figure(figsize=(10, 8))
scatter = plt.scatter(results['x'], results['y'], c=results['TrackID'], cmap='tab20', s=10)
plt.title("t-SNE visualization of merged track clusters")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(scatter, label="TrackID")
plt.show()


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Optional: Reduce dimensions using t-SNE for visualization
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_2d = tsne.fit_transform(X)

# Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=clusterer.labels_, cmap='tab20', s=10)
plt.title("TrackGNN Embedding Space with DBSCAN Clustering")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar(scatter, label="Cluster ID")
plt.grid(True)
plt.tight_layout()
plt.savefig("embedding_tsne_plot.png")
plt.show()


In [ ]:
import os
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_mean_pool

from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors
import torch.nn.functional as F


# 1. Define Model Architecture (Must match training exactly)
class EnhancedEdgeNetwork(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 128),
            torch.nn.Tanh(),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, src, dest):
        attn_weights = self.attention(torch.cat([src, dest], dim=-1))
        return attn_weights * self.edge_mlp(torch.cat([src, dest], dim=-1))

class TrackGNN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=512, embed_dim=256, n_iter=3):
        super().__init__()
        self.n_iter = n_iter
        
        # Original encoder components
        self.convs = torch.nn.ModuleList([
            GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.edge_nets = torch.nn.ModuleList([
            EnhancedEdgeNetwork(input_dim if i == 0 else hidden_dim)
            for i in range(n_iter)
        ])
        
        self.res_lin = torch.nn.ModuleList([
            torch.nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(hidden_dim) for _ in range(n_iter)
        ])
        
        self.lin = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, embed_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )

        # Add missing classifier with exact architecture from training
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2*embed_dim, 512),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(512),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(256),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, data):
        # Keep original forward logic
        x, edge_index = data.x, data.edge_index
        edge_row, edge_col = edge_index
        
        for i in range(self.n_iter):
            edge_weights = torch.sigmoid(
                self.edge_nets[i](x[edge_row], x[edge_col])
            ).squeeze()
            
            x_new = F.relu(self.bns[i](self.convs[i](x, edge_index, edge_weights)))
            x = x_new + self.res_lin[i](x)
            
        return global_mean_pool(self.lin(x), data.batch)


# 2. Dataset Class for Inference
from torch_geometric.data import Dataset

class GraphDataset(Dataset):
    def __init__(self, graph_dir):
        super().__init__()
        self.graph_dir = graph_dir
        self.file_list = sorted([f for f in os.listdir(graph_dir) if f.endswith('.pkl')])
        self.hit_ids = [self._extract_hit_id(f) for f in self.file_list]  # Store hit IDs
        self.cache = {}

    def _extract_hit_id(self, filename):
        """Extract HitID from filename (e.g., Graph_12345.pkl -> 12345)"""
        return int(filename.split('_')[1].split('.')[0])

    def load_graph(self, path):
        if path not in self.cache:
            with open(path, 'rb') as f:
                G = pickle.load(f)
            
            nodes = list(G.nodes())
            node_id_to_idx = {n: i for i, n in enumerate(nodes)}
            
            edge_list = []
            seen_edges = set()
            for u, v in G.edges():
                src = node_id_to_idx[u]
                dst = node_id_to_idx[v]
                if (src, dst) not in seen_edges and (dst, src) not in seen_edges:
                    edge_list.append([src, dst])
                    seen_edges.add((src, dst))
            
            self.cache[path] = Data(
                x=torch.tensor([G.nodes[n]['features'] for n in nodes], dtype=torch.float32),
                edge_index=torch.tensor(edge_list).t().contiguous(),
                num_nodes=len(nodes)
            )
        return self.cache[path]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        return self.load_graph(os.path.join(self.graph_dir, self.file_list[idx]))

    def get_hit_ids(self):
        """Get all HitIDs in dataset order"""
        return self.hit_ids




import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
import torch
from torch_geometric.data import Data, DataLoader


def merge_tracks(model_path, test_graph_dir, output_csv):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load original data with validation
    processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
    if 'muonID' not in processed_data.columns:
        raise ValueError("Processed data missing 'muonID' column")
    original_tracks = processed_data['muonID'].nunique()
    
    # Load model with safety checks
    checkpoint = torch.load(model_path, map_location=device)
    model = TrackGNN(
        input_dim=checkpoint['config'].get('input_dim', 4),
        hidden_dim=checkpoint['config'].get('hidden_dim', 512),
        embed_dim=checkpoint['config'].get('embed_dim', 256)
    ).to(device)
    model.load_state_dict(checkpoint['model_state'], strict=False)
    model.eval()

    # Batch processing setup
    dataset = GraphDataset(test_graph_dir)
    hit_ids = dataset.hit_ids
    
    loader = DataLoader(
        dataset,
        batch_size=64,
        num_workers=4,
        pin_memory=True
    )

    # Process graphs in batches
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing graphs"):
            batch = batch.to(device)
            embeddings.append(model(batch).cpu().numpy())
    
    X = np.concatenate(embeddings)
    X = StandardScaler().fit_transform(X)

    # Density-based clustering with updated parameters
    clusterer = DBSCAN(
        eps=0.007,        # Increased epsilon
        min_samples=2,   # Reduced minimum points
        metric='cosine',
        n_jobs=-1
    ).fit(X)
    
    # Create results without size constraints
    results = pd.DataFrame({
        'HitID': hit_ids,
        'TrackID': clusterer.labels_,
    }).merge(
        processed_data[['HitID', 'muonID']],
        on='HitID',
        how='left'
    )

    # Cluster validation without size limits
    cluster_stats = results[results['TrackID'] != -1].groupby('TrackID').agg(
        size=('HitID', 'count'),
        purity=('muonID', lambda x: x.value_counts(normalize=True).iloc[0])
    ).reset_index()

    # Remove all size constraints
    min_size = 1  # Absolute minimum
    min_purity = 0.65  # Relaxed purity threshold

    # Final validation (only purity check)
    valid_tracks = cluster_stats[
        (cluster_stats['purity'] >= min_purity)
    ]

    # Efficiency calculation
    if valid_tracks.empty:
        efficiency = 0.0
    else:
        efficiency = valid_tracks.merge(
            results[['TrackID', 'muonID']].drop_duplicates(),
            on='TrackID',
            how='left'
        )['muonID'].nunique() / original_tracks

    # Diagnostic reporting
    print("\n=== Clustering Diagnostics ===")
    print(f"Total clusters: {len(cluster_stats)}")
    print(f"Noise points: {(results['TrackID'] == -1).sum()} ({(results['TrackID'] == -1).mean():.1%})")
    print(f"Cluster size range: {cluster_stats['size'].min()}-{cluster_stats['size'].max()}")
    print(f"Validation thresholds: purity ≥ {min_purity:.2f}")
    
    print("\n=== Final Results ===")
    print(f"Original muon tracks: {original_tracks}")
    print(f"Valid reconstructed tracks: {len(valid_tracks)}")
    print(f"Tracking efficiency: {efficiency:.2%}")

    results.to_csv(output_csv, index=False)
    return results, valid_tracks, efficiency



if __name__ == "__main__":
    results, stats, efficiency = merge_tracks(
        model_path="/kaggle/input/new-set/track_gnn_best.pt",
        test_graph_dir="/kaggle/working/graphs1",
        output_csv="merged_tracks.csv"
    )

In [ ]:
import os
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_mean_pool

from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors
import torch.nn.functional as F


# 1. Define Model Architecture (Must match training exactly)
class EnhancedEdgeNetwork(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 128),
            torch.nn.Tanh(),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, src, dest):
        attn_weights = self.attention(torch.cat([src, dest], dim=-1))
        return attn_weights * self.edge_mlp(torch.cat([src, dest], dim=-1))

class TrackGNN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=512, embed_dim=256, n_iter=3):
        super().__init__()
        self.n_iter = n_iter
        
        # Original encoder components
        self.convs = torch.nn.ModuleList([
            GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.edge_nets = torch.nn.ModuleList([
            EnhancedEdgeNetwork(input_dim if i == 0 else hidden_dim)
            for i in range(n_iter)
        ])
        
        self.res_lin = torch.nn.ModuleList([
            torch.nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(hidden_dim) for _ in range(n_iter)
        ])
        
        self.lin = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, embed_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )

        # Add missing classifier with exact architecture from training
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2*embed_dim, 512),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(512),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(256),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, data):
        # Keep original forward logic
        x, edge_index = data.x, data.edge_index
        edge_row, edge_col = edge_index
        
        for i in range(self.n_iter):
            edge_weights = torch.sigmoid(
                self.edge_nets[i](x[edge_row], x[edge_col])
            ).squeeze()
            
            x_new = F.relu(self.bns[i](self.convs[i](x, edge_index, edge_weights)))
            x = x_new + self.res_lin[i](x)
            
        return global_mean_pool(self.lin(x), data.batch)


# 2. Dataset Class for Inference
from torch_geometric.data import Dataset

class GraphDataset(Dataset):
    def __init__(self, graph_dir):
        super().__init__()
        self.graph_dir = graph_dir
        self.file_list = sorted([f for f in os.listdir(graph_dir) if f.endswith('.pkl')])
        self.hit_ids = [self._extract_hit_id(f) for f in self.file_list]  # Store hit IDs
        self.cache = {}

    def _extract_hit_id(self, filename):
        """Extract HitID from filename (e.g., Graph_12345.pkl -> 12345)"""
        return int(filename.split('_')[1].split('.')[0])

    def load_graph(self, path):
        if path not in self.cache:
            with open(path, 'rb') as f:
                G = pickle.load(f)
            
            nodes = list(G.nodes())
            node_id_to_idx = {n: i for i, n in enumerate(nodes)}
            
            edge_list = []
            seen_edges = set()
            for u, v in G.edges():
                src = node_id_to_idx[u]
                dst = node_id_to_idx[v]
                if (src, dst) not in seen_edges and (dst, src) not in seen_edges:
                    edge_list.append([src, dst])
                    seen_edges.add((src, dst))
            
            self.cache[path] = Data(
                x=torch.tensor([G.nodes[n]['features'] for n in nodes], dtype=torch.float32),
                edge_index=torch.tensor(edge_list).t().contiguous(),
                num_nodes=len(nodes)
            )
        return self.cache[path]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        return self.load_graph(os.path.join(self.graph_dir, self.file_list[idx]))

    def get_hit_ids(self):
        """Get all HitIDs in dataset order"""
        return self.hit_ids




import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from tqdm.auto import tqdm
import torch
from torch_geometric.data import Data, DataLoader


def merge_tracks(model_path, test_graph_dir, output_csv):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load original data with validation
    processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
    if 'muonID' not in processed_data.columns:
        raise ValueError("Processed data missing 'muonID' column")
    original_tracks = processed_data['muonID'].nunique()
    
    # Load model with safety checks
    checkpoint = torch.load(model_path, map_location=device)
    model = TrackGNN(
        input_dim=checkpoint['config'].get('input_dim', 4),
        hidden_dim=checkpoint['config'].get('hidden_dim', 512),
        embed_dim=checkpoint['config'].get('embed_dim', 256)
    ).to(device)
    model.load_state_dict(checkpoint['model_state'], strict=False)
    model.eval()

    # Batch processing setup
    dataset = GraphDataset(test_graph_dir)
    hit_ids = dataset.hit_ids
    
    loader = DataLoader(
        dataset,
        batch_size=64,
        num_workers=4,
        pin_memory=True
    )

    # Process graphs in batches
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing graphs"):
            batch = batch.to(device)
            embeddings.append(model(batch).cpu().numpy())
    
    X = np.concatenate(embeddings)
    X = StandardScaler().fit_transform(X)

    # k-NN based track merging
    k = 3  # Number of nearest neighbors
    neighbor_model = NearestNeighbors(n_neighbors=k, metric='cosine', n_jobs=-1)
    neighbor_model.fit(X)

    # Find k nearest neighbors for each track
    distances, indices = neighbor_model.kneighbors(X)
    
    # Initialize track assignments
    track_labels = -1 * np.ones(X.shape[0], dtype=int)
    cluster_id = 0
    
    # Assign clusters based on nearest neighbors
    for i in range(len(X)):
        if track_labels[i] == -1:  # If not yet assigned
            # Assign the current node and its neighbors to the same track
            track_labels[i] = cluster_id
            for neighbor_idx in indices[i]:
                if track_labels[neighbor_idx] == -1:
                    track_labels[neighbor_idx] = cluster_id
            cluster_id += 1

    # Create results with physics constraints
    results = pd.DataFrame({
        'HitID': hit_ids,
        'TrackID': track_labels,
    }).merge(
        processed_data[['HitID', 'muonID']],
        on='HitID',
        how='left'
    )

    # Robust cluster validation
    cluster_stats = results[results['TrackID'] != -1].groupby('TrackID').agg(
        size=('HitID', 'count'),
        purity=('muonID', lambda x: x.value_counts(normalize=True).iloc[0])
    ).reset_index()

    # Handle empty/no clusters scenario
    if cluster_stats.empty:
        print("⚠️ No clusters found - all points classified as noise")
        return pd.DataFrame(), pd.DataFrame(), 0.0

    # Physics-based filtering (only purity)
    valid_tracks = cluster_stats[cluster_stats['purity'] >= 0.7]  # Fixed 70% purity threshold
    
    # Noise calculation
    noise_points = (results['TrackID'] == -1).sum()
    noise_ratio = noise_points / len(results)

    # Safe efficiency calculation
    if valid_tracks.empty:
        efficiency = 0.0
    else:
        efficiency_df = valid_tracks.merge(
            results[['TrackID', 'muonID']].drop_duplicates(),
            on='TrackID',
            how='left'
        )
        efficiency = efficiency_df['muonID'].nunique() / original_tracks

    # Diagnostic reporting
    print("\n=== Clustering Diagnostics ===")
    print(f"Total clusters: {len(cluster_stats)}")
    print(f"Noise points: {noise_points} ({noise_ratio:.1%})")
    print(f"Cluster purity range: {cluster_stats['purity'].min():.2f}-{cluster_stats['purity'].max():.2f}")
    
    print("\n=== Final Results ===")
    print(f"Original muon tracks: {original_tracks}")
    print(f"Valid reconstructed tracks (purity ≥70%): {len(valid_tracks)}")
    print(f"Tracking efficiency: {efficiency:.2%}")

    results.to_csv(output_csv, index=False)
    return results, valid_tracks, efficiency



if __name__ == "__main__":
    results, stats, efficiency = merge_tracks(
        model_path="/kaggle/input/new-set/track_gnn_maxhit.pt",
        test_graph_dir="/kaggle/working/graphs1",
        output_csv="merged_tracks.csv"
    )

In [ ]:
import os
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader  # Corrected import
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
from scipy.spatial.distance import pdist, squareform
from sklearn.neighbors import NearestNeighbors

# 1. Define Model Architecture with Edge Handling
class EnhancedEdgeNetwork(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 128),
            torch.nn.Tanh(),
            torch.nn.Linear(128, 1),
            torch.nn.Sigmoid()
        )
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(2*input_dim, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, src, dest):
        attn_weights = self.attention(torch.cat([src, dest], dim=-1))
        return attn_weights * self.edge_mlp(torch.cat([src, dest], dim=-1))

class TrackGNN(torch.nn.Module):
    def __init__(self, input_dim=4, hidden_dim=512, embed_dim=256, n_iter=3):
        super().__init__()
        self.n_iter = n_iter
        self.convs = torch.nn.ModuleList([
            GCNConv(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        self.edge_nets = torch.nn.ModuleList([
            EnhancedEdgeNetwork(input_dim if i == 0 else hidden_dim)
            for i in range(n_iter)
        ])
        self.res_lin = torch.nn.ModuleList([
            torch.nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim)
            for i in range(n_iter)
        ])
        self.bns = torch.nn.ModuleList([
            torch.nn.BatchNorm1d(hidden_dim) for _ in range(n_iter)
        ])
        self.lin = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim, embed_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2*embed_dim, 512),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(512),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.LayerNorm(256),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Handle empty edge indices
        if edge_index.size(1) == 0:
            # Process isolated nodes directly
            for i in range(self.n_iter):
                x = F.relu(self.bns[i](self.res_lin[i](x)))
            return global_mean_pool(self.lin(x), data.batch)
        
        edge_row, edge_col = edge_index
        for i in range(self.n_iter):
            edge_weights = torch.sigmoid(
                self.edge_nets[i](x[edge_row], x[edge_col])
            ).squeeze()
            x_new = F.relu(self.bns[i](self.convs[i](x, edge_index, edge_weights)))
            x = x_new + self.res_lin[i](x)
            
        return global_mean_pool(self.lin(x), data.batch)

# 2. Dataset Class with Edge Handling
class GraphDataset(Dataset):
    def __init__(self, graph_dir):
        super().__init__()
        self.graph_dir = graph_dir
        self.file_list = sorted([f for f in os.listdir(graph_dir) if f.endswith('.pkl')])
        self.hit_ids = [int(f.split('_')[1].split('.')[0]) for f in self.file_list]
        self.cache = {}

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = os.path.join(self.graph_dir, self.file_list[idx])
        if path not in self.cache:
            with open(path, 'rb') as f:
                G = pickle.load(f)
            
            nodes = list(G.nodes())
            features = np.array([G.nodes[n]['features'] for n in nodes])  # Convert to numpy first
            edge_list = []
            
            # Handle edge indices
            for u, v in G.edges():
                src = nodes.index(u)
                dst = nodes.index(v)
                if src != dst:  # Avoid self-loops
                    edge_list.append([src, dst])
                    edge_list.append([dst, src])  # Add reverse direction

            self.cache[path] = Data(
                x=torch.tensor(features, dtype=torch.float32),  # Fixed tensor creation
                edge_index=torch.tensor(edge_list).t().contiguous() if edge_list else torch.empty((2, 0), dtype=torch.long),
                num_nodes=len(nodes)
            )
        return self.cache[path]

    def get_hit_ids(self):
        return self.hit_ids

# 3. Main Processing Function
def merge_tracks(model_path, test_graph_dir, output_csv):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 1. Enhanced Data Loading with Column Verification
    processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
    
    # Verify column names exist with exact case matching
    required_columns = {'HitID', 'muonID'}
    missing_cols = required_columns - set(processed_data.columns)
    if missing_cols:
        raise KeyError(f"Missing critical columns in processed data: {missing_cols}")

    # 2. Model Loading with Architecture Verification
    checkpoint = torch.load(model_path, map_location=device, weights_only=True)
    model = TrackGNN(
        input_dim=checkpoint.get('input_dim', 4),
        hidden_dim=checkpoint.get('hidden_dim', 512),
        embed_dim=checkpoint.get('embed_dim', 256)
    ).to(device)
    model.load_state_dict(checkpoint['model_state'], strict=True)  # Enforce exact parameter match
    model.eval()

    # 3. Data Pipeline with Column Sanity Checks
    dataset = GraphDataset(test_graph_dir)
    loader = DataLoader(dataset, batch_size=64, num_workers=4, shuffle=False)

    # 4. Embedding Generation with Dimension Verification
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Processing graphs"):
            batch = batch.to(device)
            emb = model(batch)
            if emb.shape[1] != 256:  # Verify embedding dimension matches clustering needs
                raise ValueError(f"Unexpected embedding dimension: {emb.shape}")
            embeddings.append(emb.cpu().numpy())
    
    # 5. Clustering with Adaptive Parameters
    X = np.concatenate(embeddings)
    X = StandardScaler().fit_transform(X)
    
    # Dynamic epsilon calculation with bounds
    eps_percentile = np.clip(np.percentile(pdist(X), 0.000001), 1e-6, 0.1)
    dbscan = DBSCAN(
        eps=eps_percentile,
        min_samples=3,  # Increased from 3 for more stable clusters
        metric='cosine',
        n_jobs=-1
    )
    track_labels = dbscan.fit_predict(X)
    
    # 6. Result Compilation with Column Validation
    results = pd.DataFrame({
        'HitID': dataset.hit_ids,
        'TrackID': track_labels
    }).merge(
        processed_data[['HitID', 'muonID']],
        on='HitID',
        how='inner'
    )
    
    # Post-merge column verification
    if 'muonID' not in results.columns:
        missing_hits = len(dataset.hit_ids) - len(results)
        raise KeyError(f"muonID missing after merge ({missing_hits} hits lost)")

    # 7. Enhanced Physics Validation
    original_tracks = processed_data['muonID'].nunique()
    
    # Calculate noise statistics
    noise_points = (results['TrackID'] == -1).sum()
    noise_ratio = noise_points / len(results)
    
    # Cluster analysis
    valid_clusters = []
    cluster_stats = []
    
    if not results.empty:
        for track_id, group in results[results['TrackID'] != -1].groupby('TrackID'):
            muon_counts = group['muonID'].value_counts(normalize=True)
            if not muon_counts.empty:
                stats = {
                    'TrackID': track_id,
                    'size': len(group),
                    'purity': muon_counts.iloc[0],
                    'main_muon': muon_counts.idxmax()
                }
                cluster_stats.append(stats)
                
                # Apply physics criteria
                if stats['purity'] >= 0.7 and stats['size'] >= 5:
                    valid_clusters.append(stats)

    # Efficiency calculation
    valid_muons = {c['main_muon'] for c in valid_clusters}
    correctly_reconstructed = len(valid_muons)
    efficiency = correctly_reconstructed / original_tracks if original_tracks > 0 else 0

    # 8. Comprehensive Diagnostics
    print("\n=== Pipeline Diagnostics ===")
    print(f"Total input hits: {len(dataset)}")
    print(f"Original muon tracks: {original_tracks}")
    print(f"Reconstructed clusters: {results['TrackID'].nunique() - 1}")  # Exclude noise
    print(f"Noise points: {noise_points} ({noise_ratio:.2%})")
    print(f"Valid physics clusters: {len(valid_clusters)}")
    print(f"Unique muons recovered: {correctly_reconstructed}")
    print(f"Tracking efficiency: {efficiency:.2%}")
    
    if cluster_stats:
        sizes = [c['size'] for c in cluster_stats]
        purities = [c['purity'] for c in cluster_stats]
        print(f"\nAll clusters size range: {min(sizes)}-{max(sizes)}")
        print(f"All clusters purity range: {min(purities):.2f}-{max(purities):.2f}")
        
    if valid_clusters:
        valid_sizes = [c['size'] for c in valid_clusters]
        valid_purities = [c['purity'] for c in valid_clusters]
        print(f"\nValid clusters size range: {min(valid_sizes)}-{max(valid_sizes)}")
        print(f"Valid clusters avg purity: {np.mean(valid_purities):.2f}")
    else:
        print("\nNo valid clusters meeting physics criteria")

    results.to_csv(output_csv, index=False)
    return results, efficiency

if __name__ == "__main__":
    results, efficiency = merge_tracks(
        model_path="/kaggle/working/saved_models/track_gnn_20250402-100645.pt",
        test_graph_dir="/kaggle/working/graphs",
        output_csv="merged_tracks.csv"
    )

In [ ]:
graphs = pd.read_csv('/kaggle/working/merged_tracks.csv')
graphs


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm

def merge_tracks_and_evaluate(graph_dir, metadata_csv, output_csv):
    # Load metadata
    metadata = pd.read_csv(metadata_csv)
    hitid_to_muon = dict(zip(metadata['HitID'], metadata['muonID']))

    # ✅ Count total true tracks
    total_true_tracks = metadata['muonID'].nunique()

    # Get all graph files
    graph_files = [os.path.join(graph_dir, f) for f in os.listdir(graph_dir) if f.endswith('.pkl')]

    # Generate embeddings for all graphs
    embeddings = {}
    hit_ids = []

    for graph_file in tqdm(graph_files, desc="Processing graphs"):
        # Extract HitID from filename
        hit_id = int(os.path.basename(graph_file).split('_')[1].split('.')[0])
        hit_ids.append(hit_id)

        # Load and process graph
        with open(graph_file, 'rb') as f:
            G = pickle.load(f)

        # Store random 2D embeddings for DBSCAN clustering
        embeddings[hit_id] = np.random.rand(2)  

    # Convert to numpy arrays
    hit_ids = np.array(hit_ids)
    emb_array = np.array([embeddings[hid] for hid in hit_ids])

    # Cluster embeddings
    clustering = DBSCAN(eps=0.0001, min_samples=2).fit(emb_array)
    labels = clustering.labels_

    # Create result dataframe
    result_df = pd.DataFrame({
        'HitID': hit_ids,
        'PredictedTrack': labels,
        'TrueMuonID': [hitid_to_muon[hid] for hid in hit_ids]
    })

    # ✅ Sort by `TrueMuonID`
    result_df.sort_values(by='TrueMuonID', ascending=True, inplace=True)

    # ✅ Purity-based Efficiency Calculation
    reconstructed_tracks = 0
    purity_threshold = 0.7
    purity_data = []

    for true_muon in result_df['TrueMuonID'].unique():
        muon_hits = result_df[result_df['TrueMuonID'] == true_muon]

        # Count the hits per predicted track
        cluster_counts = muon_hits['PredictedTrack'].value_counts()
        
        if len(cluster_counts) == 0:
            continue

        # Identify the majority cluster
        main_cluster = cluster_counts.idxmax()
        correct_hits = cluster_counts[main_cluster]
        total_hits = len(muon_hits)

        # Calculate purity
        purity = correct_hits / total_hits

        # Only consider tracks with ≥ 70% purity
        if purity >= purity_threshold:
            reconstructed_tracks += 1

        # Store purity data
        purity_data.append({
            'TrueMuonID': true_muon,
            'MajorityCluster': main_cluster,
            'TotalHits': total_hits,
            'CorrectHits': correct_hits,
            'Purity': purity
        })

    # ✅ Calculate Overall Efficiency
    overall_efficiency = reconstructed_tracks / total_true_tracks

    print(f"\nOverall Tracking Efficiency: {overall_efficiency:.2%}")
    print(f"Number of Reconstructed Tracks (≥ 70% purity): {reconstructed_tracks}")
    print(f"Total Number of True Tracks: {total_true_tracks}")
    print(f"Number of Noise Points: {sum(labels == -1)}")

    # Save results
    result_df.to_csv(output_csv, index=False)

    # Save purity data
    purity_df = pd.DataFrame(purity_data)
    purity_df.to_csv(output_csv.replace('.csv', '_purity.csv'), index=False)

    # Save efficiency summary
    efficiency_summary = {
        'TotalTrueTracks': total_true_tracks,
        'ReconstructedTracks': reconstructed_tracks,
        'Efficiency': overall_efficiency
    }

    efficiency_df = pd.DataFrame([efficiency_summary])
    efficiency_df.to_csv(output_csv.replace('.csv', '_efficiency.csv'), index=False)

    return result_df, efficiency_df, purity_df

# ✅ Usage
if __name__ == "__main__":
    results, efficiency, purity = merge_tracks_and_evaluate(
        graph_dir="graphs",
        metadata_csv="/kaggle/working/processed_data_with_hitid.csv",
        output_csv="/kaggle/working/track_assignments.csv"
    )


In [ ]:
graphs = pd.read_csv('/kaggle/working/merged_tracks.csv')
graphs

In [ ]:
print(f"Positive samples: {sum(labels)}/{len(labels)}")
print(f"Negative samples: {len(labels)-sum(labels)}/{len(labels)}")


In [ ]:
!pip install torch==2.5.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.5.1+cu121 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.5.0+cu121.html


In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from sklearn.cluster import DBSCAN
from scipy.spatial import cKDTree
from collections import defaultdict
import networkx as nx

# Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
graphs_dir = '/kaggle/working/graphs1'
track_hits_file = '/kaggle/working/track_hits1.csv'
os.makedirs(graphs_dir, exist_ok=True)

# 1. Data Preparation
def prepare_data(processed_data):
    features = ['pos_x', 'pos_y', 'pos_z', 'time']
    normalized_data = processed_data.copy()
    for feat in features:
        min_val = processed_data[feat].min()
        max_val = processed_data[feat].max()
        normalized_data[feat] = (processed_data[feat] - min_val) / (max_val - min_val)
    return normalized_data

# 2. Global Graph Construction
class GlobalGraphBuilder:
    def __init__(self):
        self.global_graph = nx.Graph()
        self.node_mapping = {}
        self.current_idx = 0
        
    def add_subgraphs(self, graphs):
        """Add subgraphs with empty check"""
        if not graphs:
            raise ValueError("No subgraphs provided")
            
        for G in graphs:
            if len(G.nodes) == 0:
                print("Warning: Skipped empty subgraph")
                continue
        """Add subgraphs with scalar feature validation"""
        for G in graphs:
            for node, attrs in G.nodes(data=True):
                # Convert to scalar floats
                validated_attrs = {
                    'pos_x': float(attrs['pos_x']),
                    'pos_y': float(attrs['pos_y']),
                    'pos_z': float(attrs['pos_z']),
                    'time': float(attrs['time'])
                }
                if node not in self.node_mapping:
                    self.node_mapping[node] = self.current_idx
                    self.global_graph.add_node(self.current_idx, **validated_attrs)
                    self.current_idx += 1
            for u, v in G.edges():
                self.global_graph.add_edge(
                    self.node_mapping[u],
                    self.node_mapping[v],
                    edge_type='intra'
                )
                
    def add_cross_edges(self, spatial_thresh=0.1, time_thresh=0.2):
        """Propose cross-subgraph edges with dimension safety"""
        nodes = []
        for n in self.global_graph.nodes():
            attrs = self.global_graph.nodes[n]
            nodes.append([
                attrs['pos_x'], attrs['pos_y'], attrs['pos_z'], attrs['time']
            ])
        
        # Handle empty and 1D cases
        if not nodes:
            return
        nodes = np.array(nodes)
        if nodes.ndim == 1:
            nodes = nodes.reshape(1, -1)
            
        # Spatial-temporal proximity
        spatial_tree = cKDTree(nodes[:, :3])
        time_tree = cKDTree(nodes[:, 3].reshape(-1,1))
        
        spatial_pairs = spatial_tree.query_pairs(spatial_thresh)
        time_pairs = time_tree.query_pairs(time_thresh)
        valid_pairs = spatial_pairs.intersection(time_pairs)
        
        # Add edges
        for i, j in valid_pairs:
            if not self.global_graph.has_edge(i, j):
                self.global_graph.add_edge(i, j, edge_type='cross')

# 3. GNN Architecture
class TrackGNN(nn.Module):
    def __init__(self, node_dim=4, hidden_dim=64):
        super().__init__()
        self.conv1 = GATConv(node_dim, hidden_dim)
        self.conv2 = GATConv(hidden_dim, hidden_dim)
        self.edge_classifier = nn.Sequential(
            nn.Linear(2*hidden_dim + 4, hidden_dim),  # 4 = dx, dy, dz, dt
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Dimension validation
        if x.dim() != 2:
            raise ValueError(f"Input features must be 2D (got {x.dim()}D)")
            
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        
        # Edge features
        row, col = edge_index
        dx = data.x[row, 0] - data.x[col, 0]
        dy = data.x[row, 1] - data.x[col, 1]
        dz = data.x[row, 2] - data.x[col, 2]
        dt = data.x[row, 3] - data.x[col, 3]
        edge_feats = torch.stack([dx, dy, dz, dt], dim=1)
        
        # Classification
        edge_features = torch.cat([x[row], x[col], edge_feats], dim=1)
        return self.edge_classifier(edge_features).squeeze()

# 4. Training and Inference
class TrackMerger:
    def __init__(self, processor):
        self.processor = processor
        self.model = TrackGNN().to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
    
    def prepare_pyg_data(self, global_graph):
        """Create PyG data with empty check and dimension safety"""
        node_ids = sorted(global_graph.nodes())
        
        # Handle empty graph
        if not node_ids:
            raise ValueError("Global graph contains no nodes. Check input subgraphs.")
            
        # Collect node features
        node_feats = []
        for nid in node_ids:
            attrs = global_graph.nodes[nid]
            node_feats.append([
                float(attrs['pos_x']),
                float(attrs['pos_y']),
                float(attrs['pos_z']),
                float(attrs['time'])
            ])
        
        # Convert to tensor and ensure 2D
        x = torch.tensor(node_feats, dtype=torch.float32).to(device)
        if x.dim() == 1:
            x = x.unsqueeze(0)  # Convert [4] -> [1, 4]
            
        print(f"Node features shape: {x.shape}")  # Debug
        
        # Edge index handling
        edge_list = list(global_graph.edges())
        if edge_list:
            edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous().to(device)
        else:
            edge_index = torch.empty((2, 0), dtype=torch.long).to(device)
        
        return Data(x=x, edge_index=edge_index)
    
    def train(self, data, epochs=50):
        self.model.train()
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            preds = self.model(data)
            loss = F.binary_cross_entropy(preds, torch.ones_like(preds))  # Assuming all edges are valid
            loss.backward()
            self.optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    
    def predict_edges(self, data):
        self.model.eval()
        with torch.no_grad():
            return self.model(data).cpu().numpy()

# 5. Global Clustering
def merge_tracks(global_graph, edge_scores, eps=0.1):
    node_ids = list(global_graph.nodes())
    distance_matrix = np.zeros((len(node_ids), len(node_ids)))
    
    for (u, v), score in edge_scores.items():
        distance_matrix[u, v] = 1 - score
        distance_matrix[v, u] = 1 - score
    
    clusters = DBSCAN(eps=eps, min_samples=3, 
                     metric='precomputed').fit_predict(distance_matrix)
    
    tracks = {}
    for cluster_id in np.unique(clusters):
        if cluster_id == -1: continue
        indices = np.where(clusters == cluster_id)[0]
        tracks[cluster_id] = [node_ids[i] for i in indices]
    
    return tracks

# Main Workflow
if __name__ == "__main__":
    # 1. Prepare data
    processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
    processed_data = prepare_data(processed_data)
    
    # 2. Build global graph
    graphs = [pickle.load(open(os.path.join(graphs_dir, f), 'rb')) 
             for f in os.listdir(graphs_dir) if f.endswith('.pkl')]
    
    graph_builder = GlobalGraphBuilder()
    graph_builder.add_subgraphs(graphs)
    graph_builder.add_cross_edges()
    global_graph = graph_builder.global_graph
    
    # 3. Prepare GNN data
    merger = TrackMerger(graph_builder)
    pyg_data = merger.prepare_pyg_data(global_graph)
    
    # 4. Train model (or load pretrained)
    merger.train(pyg_data)  # Uncomment to train
    torch.save(merger.model.state_dict(), 'track_gnn.pth')
    merger.model.load_state_dict(torch.load('track_gnn.pth', map_location=device))
    
    # 5. Predict edge scores
    edge_scores = merger.predict_edges(pyg_data)
    edge_dict = {(u.item(), v.item()): score 
                for (u, v), score in zip(pyg_data.edge_index.t().tolist(), edge_scores)}
    
    # 6. Merge tracks
    tracks = merge_tracks(global_graph, edge_dict)
    
    # 7. Save results
    track_hits = []
    reverse_mapping = {v: k for k, v in graph_builder.node_mapping.items()}
    for track_id, nodes in tracks.items():
        original_nodes = [reverse_mapping[n] for n in nodes]
        track_hits.append({
            'track_id': track_id,
            'nodes': original_nodes,
            'num_points': len(nodes)
        })
    
    pd.DataFrame(track_hits).to_csv(track_hits_file, index=False)
    print(f"Saved {len(tracks)} merged tracks to {track_hits_file}")

In [ ]:
graphs_dir = '/kaggle/working/graphs'
updated_graphs_dir = '/kaggle/working/updated_final1'
track_hits_file = '/kaggle/working/track_hits1.csv'
graphs_hits_file = '/kaggle/working/graphs_hits.csv'
os.makedirs(updated_graphs_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

features = ['pos_z', 'time']
for feature in features:
    min_value = processed_data[feature].min()
    max_value = processed_data[feature].max()
    processed_data[feature] = (processed_data[feature] - min_value) / (max_value - min_value)

print("Min-max normalization applied to features:", features)

def load_graphs(graphs_dir):
    graphs = []
    graph_files = [f for f in os.listdir(graphs_dir) if f.endswith('.pkl')]
    for graph_file in graph_files:
        with open(os.path.join(graphs_dir, graph_file), 'rb') as f:
            G = pickle.load(f)
            graphs.append(G)
    return graphs

graphs = load_graphs(graphs_dir)

def save_graph_with_scores(graph, updated_graphs_dir, edge_scores, graph_name):
    graph_file = f'{graph_name}_with_scores.pkl'
    for (u, v), score in edge_scores.items():
        if graph.has_edge(u, v):
            graph[u][v]['score'] = score
    with open(os.path.join(updated_graphs_dir, graph_file), 'wb') as f:
        pickle.dump(graph, f)

class EdgeNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EdgeNetwork, self).__init__()
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, hi, hj):
        concatenated = torch.cat((hi, hj), dim=-1)
        return self.edge_mlp(concatenated)

class NodeNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NodeNetwork, self).__init__()
        self.node_mlp = nn.Sequential(
            nn.Linear(3 * input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, h_in, s_in, h_out):
        return self.node_mlp(torch.cat((s_in * h_in, s_in * h_out, h_in), dim=-1)) + h_in

class GNN(nn.Module):
    def __init__(self, edge_in_dim, edge_hidden_dim, edge_out_dim,
                 node_in_dim, node_hidden_dim, node_out_dim, n_iter):
        super(GNN, self).__init__()
        self.edge_network = EdgeNetwork(edge_in_dim, edge_hidden_dim, edge_out_dim)
        self.node_network = NodeNetwork(node_in_dim, node_hidden_dim, node_out_dim)
        self.n_iter = n_iter
    def forward(self, node_features, edge_indices):
        for _ in range(self.n_iter):
            new_node_features = {}
            for node_id, neighbors in edge_indices.items():
                h_in = node_features[node_id]
                s_in = []
                for neighbor_id in neighbors:
                    h_out = node_features[neighbor_id]
                    s_in.append(self.edge_network(h_in, h_out))
                if s_in:
                    s_in = torch.stack(s_in, dim=0)
                    new_node_features[node_id] = self.node_network(h_in, s_in.mean(dim=0), h_out)
                else:
                    new_node_features[node_id] = h_in
            node_features = new_node_features
        return node_features

node_input_dim = 4
model = GNN(edge_in_dim=node_input_dim,
            edge_hidden_dim=64,
            edge_out_dim=1,
            node_in_dim=node_input_dim,
            node_hidden_dim=64,
            node_out_dim=node_input_dim,
            n_iter=2)

model.to(device)
model.load_state_dict(torch.load(model_gnn, map_location=device))
model.eval()
print("Model loaded successfully.")

node_features = {}
for index, row in processed_data.iterrows():
    hit_id = row['HitID']
    pos_x = row['pos_x']
    pos_y = row['pos_y']
    pos_z = row['pos_z']
    time = row['time']
    node_features[hit_id] = torch.tensor([pos_x, pos_y, pos_z, time], dtype=torch.float32)

node_features = {k: v.to(device) for k, v in node_features.items()}
print("Node features prepared successfully.")

hitid_to_muonid = processed_data.set_index('HitID')['muonID'].to_dict()

def create_distance_matrix(edge_scores, num_nodes):
    rows = []
    cols = []
    data = []
    
    for (u, v), score in edge_scores.items():
        if u < num_nodes and v < num_nodes:
            rows.append(u)
            cols.append(v)
            data.append(1 - 2*score)  # distance = 1 - score
    
    distance_matrix = csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Symmetrize matrix
    return distance_matrix

def apply_dbscan(distance_matrix, eps=0.005):
    dbscan = DBSCAN(eps=eps, metric='precomputed', min_samples=1)
    labels = dbscan.fit_predict(distance_matrix)
    return labels

all_edge_scores = {}
track_hits = {}  # Dictionary to aggregate points and muonIDs by track label
graph_hits = []

for i, G in enumerate(graphs):
    edge_indices = {node: list(G.neighbors(node)) for node in G.nodes}
    outputs = model(node_features, edge_indices)
    edge_scores = {}
    for node_id, neighbors in edge_indices.items():
        h_in = outputs[node_id]
        for neighbor_id in neighbors:
            h_out = outputs[neighbor_id]
            edge_score = model.edge_network(h_in, h_out)
            edge_score = torch.sigmoid(edge_score).item()
            edge_scores[(node_id, neighbor_id)] = edge_score
    all_edge_scores[i] = edge_scores
    save_graph_with_scores(G, updated_graphs_dir, edge_scores, f'graph_{i + 1}')
    
    graph_hits.append({
    'track_number': i + 1,
    'points': list(G.nodes),
    'muonIDs': [hitid_to_muonid[hit_id] for hit_id in G.nodes]
    })
    
    num_nodes = len(G.nodes)
    distance_matrix = create_distance_matrix(all_edge_scores[i], num_nodes)
    labels = apply_dbscan(distance_matrix, eps=0.1)  # Adjust eps as needed
    
    # Aggregate points and muonIDs by track label
    for j, node in enumerate(G.nodes):
        label = labels[j]
        if label not in track_hits:
            track_hits[label] = {'points': [], 'muonIDs': []}
        track_hits[label]['points'].append(node)
        track_hits[label]['muonIDs'].append(hitid_to_muonid.get(node, None))

track_hits_list = [{
    'track_label': label,
    'points': data['points'],
    'muonIDs': data['muonIDs']
} for label, data in track_hits.items()]

# Print the first 5 edge scores
print("First 5 Edge Scores:")
for i, edge_scores in list(all_edge_scores.items())[:5]:
    print(f"Graph {i + 1}:")
    for (node_pair, score) in list(edge_scores.items())[:5]:
        print(f"  Edge {node_pair}: Score {score}")


track_hits_df = pd.DataFrame(track_hits_list)

track_hits_df['points'] = track_hits_df['points'].apply(lambda x: str(x))
track_hits_df['muonIDs'] = track_hits_df['muonIDs'].apply(lambda x: str(x))

graph_hits_df = pd.DataFrame(graph_hits)
graph_hits_df.to_csv(graphs_hits_file, index=False)

track_hits_df.to_csv(track_hits_file, index=False)
print(f'Track hits saved to {track_hits_file}')

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

# **🔹 Load Processed Data (for Track Labels)**
processed_data = pd.read_csv('/kaggle/working/processed_data_with_hitid.csv')
hit_to_track = dict(zip(processed_data['HitID'], processed_data['muonID']))

graph_dir = "graphs/"
graph_purities = []

# **🔹 Iterate Over Saved Graphs**
for filename in os.listdir(graph_dir):
    if filename.endswith(".pkl"):
        with open(os.path.join(graph_dir, filename), "rb") as f:
            G = pickle.load(f)
        
        # **Get Track Labels for Graph Nodes**
        hit_ids = list(G.nodes())
        track_labels = [hit_to_track[hit_id] for hit_id in hit_ids if hit_id in hit_to_track]

        if track_labels:
            # **Compute Track Counts**
            track_counts = Counter(track_labels)
            most_common_track_hits = max(track_counts.values())  # Most frequent track count
            total_hits = len(track_labels)  # Total hits in the graph
            
            purity = most_common_track_hits / total_hits
            graph_purities.append(purity)

# **🔹 Compute Average Purity**
average_purity = (np.mean(graph_purities))*100
print(f"✅ Average Graph Purity: {average_purity:.3f}%")

# **🔹 Save Purity Results**
pd.DataFrame({"Graph Purity": graph_purities}).to_csv("graph_purities.csv", index=False)


In [ ]:
graphs = pd.read_csv('/kaggle/working/track_hits.csv')
graphs

In [ ]:
graphs = pd.read_csv('/kaggle/working/track_hits.csv')
graphs

In [ ]:
### edge classification  ###

graphs_dir = '/kaggle/working/graphs1'
updated_graphs_dir = '/kaggle/working/updated_final1'
track_hits_file = '/kaggle/working/track_hits1.csv'
graphs_hits_file = '/kaggle/working/graphs_hits.csv'
os.makedirs(updated_graphs_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

features = ['pos_z', 'time']
for feature in features:
    min_value = processed_data[feature].min()
    max_value = processed_data[feature].max()
    processed_data[feature] = (processed_data[feature] - min_value) / (max_value - min_value)

print("Min-max normalization applied to features:", features)

def load_graphs(graphs_dir):
    graphs = []
    graph_files = [f for f in os.listdir(graphs_dir) if f.endswith('.pkl')]
    for graph_file in graph_files:
        with open(os.path.join(graphs_dir, graph_file), 'rb') as f:
            G = pickle.load(f)
            graphs.append(G)
    return graphs

graphs = load_graphs(graphs_dir)


def save_graph_with_scores(graph, updated_graphs_dir, edge_scores, graph_name):
    graph_file = f'{graph_name}_with_scores.pkl'
    for (u, v), score in edge_scores.items():
        if graph.has_edge(u, v):
            graph[u][v]['score'] = score
    with open(os.path.join(updated_graphs_dir, graph_file), 'wb') as f:
        pickle.dump(graph, f)

class EdgeNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EdgeNetwork, self).__init__()
        self.edge_mlp = nn.Sequential(
            nn.Linear(2 * input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, hi, hj):
        concatenated = torch.cat((hi, hj), dim=-1)
        return self.edge_mlp(concatenated)

class NodeNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NodeNetwork, self).__init__()
        self.node_mlp = nn.Sequential(
            nn.Linear(3* input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, h_in, s_in, h_out):
        return self.node_mlp(torch.cat((s_in * h_in, s_in * h_out, h_in), dim=-1)) + h_in

class GNN(nn.Module):
    def __init__(self, edge_in_dim, edge_hidden_dim, edge_out_dim,
                 node_in_dim, node_hidden_dim, node_out_dim, n_iter):
        super(GNN, self).__init__()
        self.edge_network = EdgeNetwork(edge_in_dim, edge_hidden_dim, edge_out_dim)
        self.node_network = NodeNetwork(node_in_dim, node_hidden_dim, node_out_dim)
        self.n_iter = n_iter
    def forward(self, node_features, edge_indices):
        for _ in range(self.n_iter):
            new_node_features = {}
            for node_id, neighbors in edge_indices.items():
                h_in = node_features[node_id]
                s_in = []
                for neighbor_id in neighbors:
                    h_out = node_features[neighbor_id]
                    s_in.append(self.edge_network(h_in, h_out))
                if s_in:
                    s_in = torch.stack(s_in, dim=0)
                    new_node_features[node_id] = self.node_network(h_in, s_in.mean(dim=0), h_out)
                else:
                    new_node_features[node_id] = h_in
            node_features = new_node_features
        return node_features

node_input_dim = 12
model = GNN(edge_in_dim=node_input_dim,
            edge_hidden_dim=64,
            edge_out_dim=1,
            node_in_dim=node_input_dim,
            node_hidden_dim=64,
            node_out_dim=node_input_dim,
            n_iter=2)

model.to(device)
# Load the model parameters to inspect the expected shape
checkpoint = torch.load('gnn_model.pth', map_location=device)

# Print the shape of the saved model's first layer weights
print(checkpoint['node_network.node_mlp.0.weight'].shape)

model.load_state_dict(torch.load(model_gnn, map_location=device))
model.eval()
print("Model loaded successfully.")


node_features = {}
for index, row in processed_data.iterrows():
    hit_id = row['HitID']
    pos_x = row['pos_x']
    pos_y = row['pos_y']
    pos_z = row['pos_z']
    time = row['time']
    node_features[hit_id] = torch.tensor([pos_x, pos_y, pos_z, time], dtype=torch.float32)

node_features = {k: v.to(device) for k, v in node_features.items()}
print("Node features prepared successfully.")

hitid_to_muonid = processed_data.set_index('HitID')['muonID'].to_dict()

def create_distance_matrix(edge_scores, num_nodes):
    rows = []
    cols = []
    data = []
    
    for (u, v), score in edge_scores.items():
        if u < num_nodes and v < num_nodes:
            rows.append(u)
            cols.append(v)
            data.append(1 - 2*score)  # distance = 1 - score
    
    distance_matrix = csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Symmetrize matrix
    return distance_matrix

def apply_dbscan(distance_matrix, eps=0.00005):
    dbscan = DBSCAN(eps=eps, metric='precomputed', min_samples=1)
    labels = dbscan.fit_predict(distance_matrix)
    return labels

all_edge_scores = {}
track_hits = {}  # Dictionary to aggregate points and muonIDs by track label
graph_hits = []

for i, G in enumerate(graphs):
    edge_indices = {node: list(G.neighbors(node)) for node in G.nodes}
    outputs = model(node_features, edge_indices)
    edge_scores = {}
    for node_id, neighbors in edge_indices.items():
        h_in = outputs[node_id]
        for neighbor_id in neighbors:
            h_out = outputs[neighbor_id]
            edge_score = model.edge_network(h_in, h_out)
            edge_score = torch.sigmoid(edge_score).item()
            edge_scores[(node_id, neighbor_id)] = edge_score
    all_edge_scores[i] = edge_scores
    save_graph_with_scores(G, updated_graphs_dir, edge_scores, f'graph_{i + 1}')
    
    graph_hits.append({
    'track_number': i + 1,
    'points': list(G.nodes),
    'muonIDs': [hitid_to_muonid[hit_id] for hit_id in G.nodes]
    })
    
    num_nodes = len(G.nodes)
    distance_matrix = create_distance_matrix(all_edge_scores[i], num_nodes)
    labels = apply_dbscan(distance_matrix, eps=0.1)  # Adjust eps as needed
    
    # Aggregate points and muonIDs by track label
    for j, node in enumerate(G.nodes):
        label = labels[j]
        if label not in track_hits:
            track_hits[label] = {'points': [], 'muonIDs': []}
        track_hits[label]['points'].append(node)
        track_hits[label]['muonIDs'].append(hitid_to_muonid.get(node, None))

track_hits_list = [{
    'track_label': label,
    'points': data['points'],
    'muonIDs': data['muonIDs']
} for label, data in track_hits.items()]

# Print the first 5 edge scores
print("First 5 Edge Scores:")
for i, edge_scores in list(all_edge_scores.items())[:5]:
    print(f"Graph {i + 1}:")
    for (node_pair, score) in list(edge_scores.items())[:5]:
        print(f"  Edge {node_pair}: Score {score}")


track_hits_df = pd.DataFrame(track_hits_list)

track_hits_df['points'] = track_hits_df['points'].apply(lambda x: str(x))
track_hits_df['muonIDs'] = track_hits_df['muonIDs'].apply(lambda x: str(x))

graph_hits_df = pd.DataFrame(graph_hits)
graph_hits_df.to_csv(graphs_hits_file, index=False)

track_hits_df.to_csv(track_hits_file, index=False)
print(f'Track hits saved to {track_hits_file}')

In [ ]:
graphs = pd.read_csv('/kaggle/working/track_hits.csv')
graphs

In [ ]:
# Manually set the correct CUDA paths
import os

# Kaggle's CUDA 11.8 path
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.8'
os.environ['PATH'] = '/usr/local/cuda-11.8/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:' + os.environ.get('LD_LIBRARY_PATH', '')

# Verify CUDA paths
!echo $CUDA_HOME
!nvcc --version


In [ ]:
!pip uninstall torch torch-geometric pyg-lib -y
